<a href="https://colab.research.google.com/github/Deusxy/Ribosome-Binding-Site-Calculator-v1.0/blob/master/RBSCal_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 
Side project get the RBS calculater runing as well as the disign. 
https://github.com/Deusxy/Ribosome-Binding-Site-Calculator-v1.0 

In [ ]:
#@title code of ViennaRNA
#This file is part of the Ribosome Binding Site Calculator.
#The Ribosome Binding Site Calculator is free software: you can redistribute it and/or modify

import os.path
import os, popen2, time

current_dir = os.path.dirname(os.path.abspath(__file__)) + "/tmp"
if not os.path.exists(current_dir): os.mkdir(current_dir)

debug=0

#Class that encapsulates all of the functions from NuPACK 2.0
class ViennaRNA(dict):

    debug_mode = 0
    RT = 0.61597 #gas constant times 310 Kelvin (in units of kcal/mol)

    def __init__(self,Sequence_List,material = "rna37"):

        self.ran = 0

        import re
        import random
        import string

        exp = re.compile('[ATGCU]',re.IGNORECASE)

        for seq in Sequence_List:
            if exp.match(seq) == None:
                error_string = "Invalid letters found in inputted sequences. Only ATGCU allowed. \n Sequence is \"" + str(seq) + "\"."
                raise ValueError(error_string)

        self["sequences"] = Sequence_List
        self["material"] = material

        random.seed(time.time())
        long_id = "".join([random.choice(string.letters + string.digits) for x in range(10)])
        self.prefix = current_dir + "/temp_" + long_id

    def mfe(self, strands,Temp = 37.0, dangles = "all",outputPS = False):

        self["mfe_composition"] = strands

        if Temp <= 0: raise ValueError("The specified temperature must be greater than zero.")

        seq_string = "&".join(self["sequences"])
        input_string = seq_string + "\n"

        handle = open(self.prefix,"w")
        handle.write(input_string)
        handle.close()

        #Set arguments
        material = self["material"]
        if dangles is "none":
            dangles = " -d0 "
        elif dangles is "some":
            dangles = " -d1 "
        elif dangles is "all":
            dangles = " -d2 "

        if outputPS:
            outputPS_str = " "
        else:
            outputPS_str = " -noPS "

        #Call ViennaRNA C programs
        cmd = "RNAcofold"
        args = outputPS_str + dangles + " < " + self.prefix

        output = popen2.Popen3(cmd + args)
        #output.tochild.write(input_string)

        while output.poll() < 0:
            try:
                output.wait()
                time.sleep(0.001)
            except:
                break

        if debug == 1: print output.fromchild.read()

        #Skip the unnecessary output lines
        line = output.fromchild.readline()

        line = output.fromchild.readline()
        words = line.split(" ")
        bracket_string = words[0]
        (strands,bp_x, bp_y) = self.convert_bracket_to_numbered_pairs(bracket_string)

        energy = float(words[len(words)-1].replace(")","").replace("(","").replace("\n",""))

        self._cleanup()
        self["program"] = "mfe"
        self["mfe_basepairing_x"] = [bp_x]
        self["mfe_basepairing_y"] = [bp_y]
        self["mfe_energy"] = [energy]
        self["totalnt"]=strands


        #print "Minimum free energy secondary structure has been calculated."

    def subopt(self, strands,energy_gap,Temp = 37.0, dangles = "all", outputPS = False):

        self["subopt_composition"] = strands

        if Temp <= 0: raise ValueError("The specified temperature must be greater than zero.")

        seq_string = "&".join(self["sequences"])
        input_string = seq_string + "\n"

        handle = open(self.prefix,"w")
        handle.write(input_string)
        handle.close()

        #Set arguments
        material = self["material"]
        if dangles is "none":
            dangles = " -d0 "
        elif dangles is "some":
            dangles = " -d1 "
        elif dangles is "all":
            dangles = " -d2 "

        if outputPS:
            outputPS_str = " "
        else:
            outputPS_str = " -noPS "

        #Call ViennaRNA C programs
        cmd = "RNAsubopt"
        args = " -e " + str(energy_gap) + outputPS_str + dangles + " < " + self.prefix

        output = popen2.Popen3(cmd + args)

        while output.poll() < 0:
            try:
                output.wait()
                time.sleep(0.001)
            except:
                break

        #print output.fromchild.read()
        if debug == 1: print output.fromchild.read()

        #Skip unnecessary line
        line = output.fromchild.readline()

        self["subopt_basepairing_x"] = []
        self["subopt_basepairing_y"] = []
        self["subopt_energy"] = []
        self["totalnt"]=[]
        counter=0

        while len(line)>0:
            line = output.fromchild.readline()
            if len(line) > 0:
                counter+=1
                words = line.split(" ")
                bracket_string = words[0]
                energy = float(words[len(words)-1].replace("\n",""))

                (strands,bp_x, bp_y) = self.convert_bracket_to_numbered_pairs(bracket_string)

                self["subopt_energy"].append(energy)
                self["subopt_basepairing_x"].append(bp_x)
                self["subopt_basepairing_y"].append(bp_y)

        self["subopt_NumStructs"] = counter

        self._cleanup()
        self["program"] = "subopt"

        #print "Minimum free energy and suboptimal secondary structures have been calculated."

    def energy(self, strands, base_pairing_x, base_pairing_y, Temp = 37.0, dangles = "all"):

        self["energy_composition"] = strands

        if Temp <= 0: raise ValueError("The specified temperature must be greater than zero.")

        seq_string = "&".join(self["sequences"])
        strands = [len(seq) for seq in self["sequences"]]
        bracket_string = self.convert_numbered_pairs_to_bracket(strands,base_pairing_x,base_pairing_y)
        input_string = seq_string + "\n" + bracket_string + "\n"

        handle = open(self.prefix,"w")
        handle.write(input_string)
        handle.close()

        #Set arguments
        material = self["material"]
        if dangles is "none":
            dangles = " -d0 "
        elif dangles is "some":
            dangles = " -d1 "
        elif dangles is "all":
            dangles = " -d2 "

        #Call ViennaRNA C programs
        cmd = "RNAeval"
        args = dangles + " < " + self.prefix

        output = popen2.Popen3(cmd + args)

        while output.poll() < 0:
            try:
                output.wait()
                time.sleep(0.001)
            except:
                break

        #if debug == 1: print output.fromchild.read()

        self["energy_energy"] = []

        #Skip the unnecessary output lines
        line = output.fromchild.readline()

        line = output.fromchild.readline()
        words = line.split(" ")
        energy = float(words[len(words)-1].replace("(","").replace(")","").replace("\n",""))

        self["program"] = "energy"
        self["energy_energy"].append(energy)
        self["energy_basepairing_x"] = [base_pairing_x]
        self["energy_basepairing_y"] = [base_pairing_y]
        self._cleanup()

        return energy

    def convert_bracket_to_numbered_pairs(self,bracket_string):

        all_seq_len = len(bracket_string)
        bp_x = []
        bp_y = []
        strands = []

        for y in range(bracket_string.count(")")):
            bp_y.append([])

        last_nt_x_list = []
        counter=0
        num_strands=0
        for (pos,letter) in enumerate(bracket_string[:]):
            if letter is ".":
                counter += 1

            elif letter is "(":
                bp_x.append(pos-num_strands)
                last_nt_x_list.append(pos-num_strands)
                counter += 1

            elif letter is ")":
                nt_x = last_nt_x_list.pop()
                nt_x_pos = bp_x.index(nt_x)
                bp_y[nt_x_pos] = pos-num_strands
                counter += 1

            elif letter is "&":
                strands.append(counter)
                counter=0
                num_strands+=1

            else:
                print "Error! Invalid character in bracket notation."

        if len(last_nt_x_list) > 0:
            print "Error! Leftover unpaired nucleotides when converting from bracket notation to numbered base pairs."

        strands.append(counter)
        bp_x = [pos+1 for pos in bp_x[:]] #Shift so that 1st position is 1
        bp_y = [pos+1 for pos in bp_y[:]] #Shift so that 1st position is 1

        return (strands,bp_x, bp_y)

    def convert_numbered_pairs_to_bracket(self,strands,bp_x,bp_y):

        bp_x = [pos-1 for pos in bp_x[:]] #Shift so that 1st position is 0
        bp_y = [pos-1 for pos in bp_y[:]] #Shift so that 1st position is 0

        bracket_notation = []
        counter=0
        for (strand_number,seq_len) in enumerate(strands):
            if strand_number > 0: bracket_notation.append("&")
            for pos in range(counter,seq_len+counter):
                if pos in bp_x:
                    bracket_notation.append("(")
                elif pos in bp_y:
                    bracket_notation.append(")")
                else:
                    bracket_notation.append(".")
            counter+=seq_len

        return "".join(bracket_notation)

    def _cleanup(self):

        if os.path.exists(self.prefix): os.remove(self.prefix)
        return

if __name__ == "__main__":

    sequences = ["AGGGGGGATCTCCCCCCAAAAAATAAGAGGTACACATGACTAAAACTTTCAAAGGCTCAGTATTCCCACT"] #,"acctcctta"]
    test = ViennaRNA(sequences,material = "rna37")

    test.mfe([1],Temp = 37.0, dangles = "all")

    bp_x = test["mfe_basepairing_x"][0]
    bp_y = test["mfe_basepairing_y"][0]
    strands = test["totalnt"]
    bracket_string = test.convert_numbered_pairs_to_bracket(strands,bp_x,bp_y)
    print bracket_string

    (strands,bp_x, bp_y) = test.convert_bracket_to_numbered_pairs(bracket_string)

    print "Strands = ", strands
    print "bp_x = ", bp_x
    print "bp_y = ", bp_y

    print test.energy(strands, bp_x, bp_y, dangles = "all")
    test.subopt(strands,3.5,dangles = "all")
    print test

#    print bracket_string
#    print test.convert_numbered_pairs_to_bracket(strands,bp_x,bp_y)


SyntaxError: ignored

In [ ]:
#@title code of NUPACK
#Python wrapper for NUPACK 2.0 by Dirks, Bois, Schaeffer, Winfree, and Pierce (SIAM Review)

#This file is part of the Ribosome Binding Site Calculator.

import os.path
import os, popen2, time, random, string

tempdir = "/tmp" + "".join([random.choice(string.digits) for x in range(6)])

current_dir = os.path.dirname(os.path.abspath(__file__)) + tempdir
if not os.path.exists(current_dir): os.mkdir(current_dir)

debug=0

#Class that encapsulates all of the functions from NuPACK 2.0
class NuPACK(dict):

    debug_mode = 0
    RT = 0.61597 #gas constant times 310 Kelvin (in units of kcal/mol)

    def __init__(self,Sequence_List,material):

        self.ran = 0

        import re
        import string

        exp = re.compile('[ATGCU]',re.IGNORECASE)

        for seq in Sequence_List:
            if exp.match(seq) == None:
                error_string = "Invalid letters found in inputted sequences. Only ATGCU allowed. \n Sequence is \"" + str(seq) + "\"."
                raise ValueError(error_string)

        if not material == 'rna' and not material == 'dna' and not material == "rna1999": raise ValueError("The energy model must be specified as either ""dna"", ""rna"", or ""rna1999"" .")

        self["sequences"] = Sequence_List
        self["material"] = material

        random.seed(time.time())
        long_id = "".join([random.choice(string.letters + string.digits) for x in range(10)])
        self.prefix = current_dir + "/nu_temp_" + long_id

    def complexes(self,MaxStrands, Temp = 37.0, ordered = "", pairs = "", mfe = "", degenerate = "", dangles = "some", timeonly = "", quiet="", AdditionalComplexes = []):
        """A wrapper for the complexes command, which calculates the equilibrium probability of the formation of a multi-strand
        RNA or DNA complex with a user-defined maximum number of strands.  Additional complexes may also be included by the user."""

        if Temp <= 0: raise ValueError("The specified temperature must be greater than zero.")
        if int(MaxStrands) <= 0: raise ValueError("The maximum number of strands must be greater than zero.")

        #Write input files
        self._write_input_complexes(MaxStrands, AdditionalComplexes)

        #Set arguments
        material = self["material"]
        if ordered: ordered = " -ordered "
        if pairs: pairs = " -pairs "
        if mfe: mfe = " -mfe "
        if degenerate: degenerate = " -degenerate "
        if timeonly: timeonly = " -timeonly "
        if quiet: quiet = " -quiet "
        dangles = "-dangles " + dangles + " "


        #Call NuPACK C programs
        cmd = "complexes"
        args = " -T " + str(Temp) + " -material " + material + " " + ordered + pairs + mfe + degenerate \
        + dangles + timeonly + quiet + " "

        output = popen2.Popen3(cmd + args + self.prefix)
        while output.poll() < 0:
            try:
                output.wait()
                time.sleep(0.001)
            except:
                break

        if debug == 1: print output.fromchild.read()

        #Read output files
        self._read_output_cx()
        self._cleanup("cx")


        if ordered:
            self._read_output_ocx()
            self._read_output_ocx_mfe()

            self._cleanup("ocx")
            self._cleanup("ocx-mfe")
            self._cleanup("ocx-key")
        self._cleanup("in")

        #print "Complex energies and secondary structures calculated."
        self.ran = 1
        self["program"] = "complexes"

    def mfe(self, strands,Temp = 37.0, multi = " -multi", pseudo = "", degenerate = "", dangles = "some"):

        self["mfe_composition"] = strands

        if Temp <= 0: raise ValueError("The specified temperature must be greater than zero.")

        if (multi == 1 and pseudo == 1): raise ValueError("The pseudoknot algorithm does not work with the -multi option.")

        #Write input files
        self._write_input_mfe(strands)

        #Set arguments
        material = self["material"]
        if multi == "": multi = ""
        if pseudo: pseudo = " -pseudo"
        if degenerate: degenerate = " -degenerate "
        dangles = " -dangles " + dangles + " "

        #Call NuPACK C programs
        cmd = "mfe"
        args = " -T " + str(Temp) + multi + pseudo + " -material " + material + degenerate + dangles + " "

        output = popen2.Popen3(cmd + args + self.prefix)
        while output.poll() < 0:
            try:
                output.wait()
                time.sleep(0.001)
            except:
                break

        if debug == 1: print output.fromchild.read()

        self._read_output_mfe()
        self._cleanup("mfe")
        self._cleanup("in")
        self["program"] = "mfe"
        #print "Minimum free energy secondary structure has been calculated."


    def subopt(self, strands,energy_gap,Temp = 37.0, multi = " -multi", pseudo = "", degenerate = "", dangles = "some"):

        self["subopt_composition"] = strands

        if Temp <= 0: raise ValueError("The specified temperature must be greater than zero.")

        if (multi == 1 and pseudo == 1): raise ValueError("The pseudoknot algorithm does not work with the -multi option.")

        #Write input files
        self._write_input_subopt(strands,energy_gap)

        #Set arguments
        material = self["material"]
        if multi == "": multi = ""
        if pseudo: pseudo = " -pseudo"
        if degenerate: degenerate = " -degenerate "
        dangles = " -dangles " + dangles + " "

        #Call NuPACK C programs
        cmd = "subopt"
        args = " -T " + str(Temp) + multi + pseudo + " -material " + material + degenerate + dangles + " "

        output = popen2.Popen3(cmd + args + self.prefix)
        while output.poll() < 0:
            try:
                output.wait()
                time.sleep(0.001)
            except:
                break

        if debug == 1: print output.fromchild.read()

        self._read_output_subopt()
        self._cleanup("subopt")
        self._cleanup("in")
        self["program"] = "subopt"

        #print "Minimum free energy and suboptimal secondary structures have been calculated."

    def energy(self, strands, base_pairing_x, base_pairing_y, Temp = 37.0, multi = " -multi", pseudo = "", degenerate = "", dangles = "some"):

        self["energy_composition"] = strands

        if Temp <= 0: raise ValueError("The specified temperature must be greater than zero.")

        if (multi == 1 and pseudo == 1): raise ValueError("The pseudoknot algorithm does not work with the -multi option.")

        #Write input files
        self._write_input_energy(strands,base_pairing_x,base_pairing_y)

        #Set arguments
        material = self["material"]
        if multi == "": multi = ""
        if pseudo: pseudo = " -pseudo"
        if degenerate: degenerate = " -degenerate "
        dangles = " -dangles " + dangles + " "

        #Call NuPACK C programs
        cmd = "energy"
        args = " -T " + str(Temp) + multi + pseudo + " -material " + material + degenerate + dangles + " "

        output = popen2.Popen3(cmd + args + self.prefix)
        while output.poll() < 0:
            try:
                output.wait()
                time.sleep(0.001)
            except:
                break

        #if debug == 1: print output.fromchild.read()

        self["energy_energy"] = []

        #Skip the comments of the text file
        line = output.fromchild.readline()
        while line[0]=="%":
            line = output.fromchild.readline()


        energy = float(line)
        self["program"] = "energy"
        self["energy_energy"].append(energy)
        self["energy_basepairing_x"] = [base_pairing_x]
        self["energy_basepairing_y"] = [base_pairing_y]
        self._cleanup("in")

        return energy

    def pfunc(self, strands, Temp = 37.0, multi = " -multi", pseudo = "", degenerate = "", dangles = "some"):

        self["pfunc_composition"] = strands

        if Temp <= 0: raise ValueError("The specified temperature must be greater than zero.")

        if (multi == 1 and pseudo == 1): raise ValueError("The pseudoknot algorithm does not work with the -multi option.")

        #Write input files
        #Input for pfunc is the same as mfe
        self._write_input_mfe(strands)

        #Set arguments
        material = self["material"]
        if multi == "": multi = ""
        if pseudo: pseudo = " -pseudo"
        if degenerate: degenerate = " -degenerate "
        dangles = " -dangles " + dangles + " "

        #Call NuPACK C programs
        cmd = "pfunc"
        args = " -T " + str(Temp) + multi + pseudo + " -material " + material + degenerate + dangles + " "

        output = popen2.Popen3(cmd + args + self.prefix)
        while output.poll() < 0:
            try:
                output.wait()
                time.sleep(0.001)
            except:
                break

        #if debug == 1: print output.fromchild.read()

        #Skip the comments of the text file
        line = output.fromchild.readline()
        words = line.split(" ")
        while line[0]=="%" or words[0] == "Attempting":
            line = output.fromchild.readline()
            words = line.split(" ")

        energy = float(line)

        line = output.fromchild.readline()
        partition_function = float(line)

        self["program"] = "pfunc"
        self["pfunc_energy"] = energy
        self["pfunc_partition_function"] = partition_function
        self._cleanup("in")

        return partition_function

    def count(self, strands, Temp = 37.0, multi = " -multi", pseudo = "", degenerate = "", dangles = "some"):

        self["count_composition"] = strands

        if (multi == 1 and pseudo == 1): raise ValueError("The pseudoknot algorithm does not work with the -multi option.")

        #Write input files
        #Input for count is the same as mfe
        self._write_input_mfe(strands)

        #Set arguments
        material = self["material"]
        if multi == "": multi = ""
        if pseudo: pseudo = " -pseudo"
        if degenerate: degenerate = " -degenerate "
        dangles = " -dangles " + dangles + " "

        #Call NuPACK C programs
        cmd = "count"
        args = " -T " + str(Temp) + multi + pseudo + " -material " + material + degenerate + dangles + " "

        output = popen2.Popen3(cmd + args + self.prefix)
        while output.poll() < 0:
            try:
                output.wait()
                time.sleep(0.001)
            except:
                break

        #if debug == 1: print output.fromchild.read()

        #Skip the comments of the text file
        line = output.fromchild.readline()
        words = line.split(" ")
        while line[0]=="%" or words[0] == "Attempting":
            line = output.fromchild.readline()
            words = line.split(" ")

        number = float(line)

        self["program"] = "count"
        self["count_number"] = number
        self._cleanup("in")

        return number

    def _write_input_energy(self,strands,base_pairing_x,base_pairing_y):
        #Creates the input file for energy NUPACK functions
        #strands is a list containing the number of each strand in the complex (assumes -multi flag is used)
        #base_pairing_x and base_pairing_y is a list of base pairings of the strands s.t. #x < #y are base paired

        NumStrands = len(self["sequences"])
        input_str = str(NumStrands) + "\n"
        for seq in self["sequences"]:
            input_str = input_str + seq + "\n"

        NumEachStrands = ""
        for num in strands:
            NumEachStrands = NumEachStrands + str(num) + " "

        input_str = input_str + NumEachStrands + "\n"
        for pos in range(len(base_pairing_x)):
            input_str = input_str + str(base_pairing_x[pos]) + "\t" + str(base_pairing_y[pos]) + "\n"

        handle = open(self.prefix + ".in", "w")
        handle.writelines(input_str)
        handle.close()

    def _write_input_subopt(self,strands,energy_gap):
        #Creates the input file for mfe and subopt NUPACK functions
        #strands is a list containing the number of each strand in the complex (assumes -multi flag is used)

        NumStrands = len(self["sequences"])
        input_str = str(NumStrands) + "\n"
        for seq in self["sequences"]:
            input_str = input_str + seq + "\n"

        NumEachStrands = ""
        for num in strands:
            NumEachStrands = NumEachStrands + str(num) + " "

        input_str = input_str + NumEachStrands + "\n"
        input_str = input_str + str(energy_gap) + "\n"

        handle = open(self.prefix + ".in", "w")
        handle.writelines(input_str)
        handle.close()

    def _write_input_mfe(self,strands):
        #Creates the input file for mfe and subopt NUPACK functions
        #strands is a list containing the number of each strand in the complex (assumes -multi flag is used)

        NumStrands = len(self["sequences"])
        input_str = str(NumStrands) + "\n"
        for seq in self["sequences"]:
            input_str = input_str + seq + "\n"

        NumEachStrands = ""
        for num in strands:
            NumEachStrands = NumEachStrands + str(num) + " "

        input_str = input_str + NumEachStrands + "\n"

        handle = open(self.prefix + ".in", "w")
        handle.writelines(input_str)
        handle.close()

    def _write_input_complexes(self, MaxStrands, AdditionalComplexes = [] ):

        #First, create the input string for file.in to send into NUPACK
        NumStrands = len(self["sequences"])
        input_str = str(NumStrands) + "\n"
        for seq in self["sequences"]:
            input_str = input_str + seq + "\n"
        input_str = input_str + str(MaxStrands) + "\n"

        handle = open(self.prefix + ".in", "w")
        handle.writelines(input_str)
        handle.close()

        if len(AdditionalComplexes) > 0:
            #The user may also specify additional complexes composed of more than MaxStrands strands. Create the input string detailing this.
            counter=0
            counts = [[]]
            added=[]
            for (complexes,i) in zip(AdditionalComplexes,range(len(AdditionalComplexes))):

                if len(complexes) <= MaxStrands: #Remove complexes if they have less than MaxStrands strands.
                    AdditionalComplexes.pop(i)
                else:
                    counts.append([])
                    added.append(0)
                    for j in range(NumStrands): #Count the number of each unique strand in each complex and save it to counts
                        counts[counter].append(complexes.count(j+1))
                    counter += 1

            list_str = ""
            for i in range(len(counts)-1):
                if added[i] == 0:
                    list_str = list_str + "C " + " ".join([str(count) for count in counts[i]]) + "\n"
                    list_str = list_str + " ".join([str(strand) for strand in AdditionalComplexes[i]]) + "\n"
                    added[i] = 1
                    for j in range(i+1,len(counts)-1):
                        if counts[i] == counts[j] and added[j] == 0:
                            list_str = list_str + " ".join([str(strand) for strand in AdditionalComplexes[j]]) + "\n"
                            added[j]=1

            handle = open(self.prefix + ".list", "w")
            handle.writelines(list_str)
            handle.close()

    def _read_output_cx(self):

        #Read the prefix.cx output text file generated by NuPACK and write its data to instanced attributes
    #Output: energies of unordered complexes in key "unordered_energies"
    #Output: strand composition of unordered complexes in key "unordered_complexes"

        handle = open(self.prefix+".cx", "rU")

        line = handle.readline()

        #Read some useful data from the comments of the text file
        while line[0]=="%":

            words=line.split()

            if len(words) > 7 and words[1] == "Number" and words[2] == "of" and words[3] == "complexes" and words[4] == "from" and words[5] == "enumeration:":
                self["numcomplexes"] =int(words[6])

            elif len(words) > 8 and words[1] == "Total" and words[2] == "number" and words[3] == "of" and words[4] =="permutations" and words[5] == "to" and words[6] == "calculate:":
                self["num_permutations"] = int(words[7])

            line = handle.readline()

        self["unordered_energies"] = []
        self["unordered_complexes"] = []
        self["unordered_composition"] = []

        while line:
            words=line.split()

            if not words[0] == "%":

                complex = words[0]
                strand_compos = [int(f) for f in words[1:len(words)-1]]
                energy = float(words[len(words)-1])

                self["unordered_complexes"].append(complex)
                self["unordered_energies"].append(energy)
                self["unordered_composition"].append(strand_compos)

            line = handle.readline()
        handle.close()

    def _read_output_ocx(self):

    #Read the prefix.ocx output text file generated by NuPACK and write its data to instanced attributes
    #Output: energies of ordered complexes in key "ordered_energies"
    #Output: number of permutations and strand composition of ordered complexes in key "ordered_complexes"

        handle = open(self.prefix+".ocx", "rU")

        line = handle.readline()

        #Read some useful data from the comments of the text file
        while line[0]=="%":

            words=line.split()

            if len(words) > 7 and words[1] == "Number" and words[2] == "of" and words[3] == "complexes" and words[4] == "from" and words[5] == "enumeration:":
                self["numcomplexes"] =int(words[6])

            elif len(words) > 8 and words[1] == "Total" and words[2] == "number" and words[3] == "of" and words[4] =="permutations" and words[5] == "to" and words[6] == "calculate:":
                self["num_permutations"] = int(words[7])

            line = handle.readline()

        self["ordered_complexes"] = []
        self["ordered_energies"] = []
        self["ordered_permutations"] = []
        self["ordered_composition"] = []


        while line:
            words=line.split()

            if not words[0] == "%":

                complex = words[0]
                permutations = words[1]
                strand_compos = [int(f) for f in words[2:len(words)-1]]
                energy = float(words[len(words)-1])

                self["ordered_complexes"].append(complex)
                self["ordered_permutations"].append(permutations)
                self["ordered_energies"].append(energy)
                self["ordered_composition"].append(strand_compos)

            line = handle.readline()
        handle.close()

    def _read_output_ocx_mfe(self):
    #Read the prefix.ocx output text file generated by NuPACK and write its data to instanced attributes
    #Output: energy of mfe of each complex in key "ordered_energy"


        #Make sure that the ocx file has already been read.
        if not (self.has_key("ordered_complexes") and self.has_key("ordered_permutations") and self.has_key("ordered_energies") and self.has_key("ordered_composition")):
            self._read_output_ocx(self,prefix)

        handle = open(self.prefix+".ocx-mfe", "rU")

        #Skip the comments of the text file
        line = handle.readline()
        while line[0]=="%":
            line = handle.readline()

        self["ordered_basepairing_x"] = []
        self["ordered_basepairing_y"] = []
        self["ordered_energy"] = []
        self["ordered_totalnt"]=[]

        while line:
            words=line.split()

            if not line == "\n" and not words[0] == "%" and not words[0] == "":

                #Read the line containing the number of total nucleotides in the complex
                totalnt = words[0]

                self["ordered_totalnt"].append(totalnt)

                #Read the line containing the mfe
                words = handle.readline().split()
                mfe = float(words[0])

                self["ordered_energy"].append(mfe)

                #Skip the line containing the dot/parens description of the secondary structure
                line = handle.readline()

                #Read in the lines containing the base pairing description of the secondary structure
                #Continue reading until a % comment
                bp_x = []
                bp_y = []

                line = handle.readline()
                words = line.split()
                while not line == "\n" and not words[0] == "%":
                    bp_x.append(int(words[0]))
                    bp_y.append(int(words[1]))
                    words = handle.readline().split()

                self["ordered_basepairing_x"].append(bp_x)
                self["ordered_basepairing_y"].append(bp_y)

            line = handle.readline()
        handle.close()

    def _read_output_mfe(self):
    #Read the prefix.mfe output text file generated by NuPACK and write its data to instanced attributes
    #Output: total sequence length and minimum free energy
    #Output: list of base pairings describing the secondary structure

        handle = open(self.prefix+".mfe", "rU")

        #Skip the comments of the text file
        line = handle.readline()
        while line[0]=="%":
            line = handle.readline()

        self["mfe_basepairing_x"] = []
        self["mfe_basepairing_y"] = []
        self["mfe_energy"] = []
        self["totalnt"]=[]

        counter = 0
        while line:
            words=line.split()

            if not line == "\n" and not words[0] == "%" and not words[0] == "":

                #Read the line containing the number of total nucleotides in the complex
                totalnt = words[0]

                self["totalnt"].append(totalnt)
                counter += 1

                #Read the line containing the mfe
                words = handle.readline().split()
                mfe = float(words[0])

                self["mfe_energy"].append(mfe)

                #Skip the line containing the dot/parens description of the secondary structure
                line = handle.readline()

                #Read in the lines containing the base pairing description of the secondary structure
                #Continue reading until a % comment
                bp_x = []
                bp_y = []

                line = handle.readline()
                words = line.split()
                while not line == "\n" and not words[0] == "%":
                    bp_x.append(int(words[0]))
                    bp_y.append(int(words[1]))
                    words = handle.readline().split()

                self["mfe_basepairing_x"].append(bp_x)
                self["mfe_basepairing_y"].append(bp_y)

            line = handle.readline()
        handle.close()

        self["mfe_NumStructs"] = counter

    def _read_output_subopt(self):
    #Read the prefix.subopt output text file generated by NuPACK and write its data to instanced attributes
    #Output: total sequence length and minimum free energy
    #Output: list of base pairings describing the secondary structure

        handle = open(self.prefix+".subopt", "rU")

        #Skip the comments of the text file
        line = handle.readline()
        while line[0]=="%":
            line = handle.readline()

        self["subopt_basepairing_x"] = []
        self["subopt_basepairing_y"] = []
        self["subopt_energy"] = []
        self["totalnt"]=[]

        counter=0

        while line:
            words=line.split()

            if not line == "\n" and not words[0] == "%" and not words[0] == "":

                #Read the line containing the number of total nucleotides in the complex
                totalnt = words[0]

                self["totalnt"].append(totalnt)
                counter += 1

                #Read the line containing the mfe
                words = handle.readline().split()
                mfe = float(words[0])

                self["subopt_energy"].append(mfe)

                #Skip the line containing the dot/parens description of the secondary structure
                line = handle.readline()

                #Read in the lines containing the base pairing description of the secondary structure
                #Continue reading until a % comment
                bp_x = []
                bp_y = []

                line = handle.readline()
                words = line.split()
                while not line == "\n" and not words[0] == "%":
                    bp_x.append(int(words[0]))
                    bp_y.append(int(words[1]))
                    words = handle.readline().split()

                self["subopt_basepairing_x"].append(bp_x)
                self["subopt_basepairing_y"].append(bp_y)

            line = handle.readline()
        handle.close()

        self["subopt_NumStructs"] = counter

    def _cleanup(self,suffix):

        if os.path.exists(self.prefix+"."+suffix): os.remove(self.prefix+"."+suffix)

        return

    def export_PDF(self, complex_ID, name = "", filename = "temp.pdf", program = None):
        """Uses Zuker's sir_graph_ng and ps2pdf.exe to convert a secondary structure described in .ct format
        to a PDF of the RNA"""

        if program is None:
            program = self["program"]

        inputfile = "temp.ct"
        self.Convert_to_ct(complex_ID,name,inputfile,program)


        cmd = "sir_graph_ng" #Assumes it's on the path
        args = "-p" #to PostScript file
        output = popen2.Popen3(cmd + " " + args + " " + inputfile,"r")
        output.wait()
        if debug == 1: print output.fromchild.read()

        inputfile = inputfile[0:len(inputfile)-2] + "ps"

        cmd = "ps2pdf" #Assumes it's on the path
        output = popen2.Popen3(cmd + " " + inputfile,"r")
        output.wait()
        if debug == 1: print output.fromchild.read()

        outputfile = inputfile[0:len(inputfile)-2] + "pdf"

        #Remove the temporary file "temp.ct" if it exists
        if os.path.exists("temp.ct"): os.remove("temp.ct")

        #Remove the temporary Postscript file if it exists
        if os.path.exists(inputfile): os.remove(inputfile)

        #Rename the output file to the desired filename.
        if os.path.exists(outputfile): os.rename(outputfile,filename)
        #Done!

    def Convert_to_ct(self,complex_ID,name,filename = "temp.ct",program = "ordered"):
        """Converts the secondary structure of a single complex into the .ct file format, which is used
        with sir_graph_ng (or other programs) to create an image of the secondary structure."""

        #hacksy way of reading from data produced by 'complex', by 'mfe', or by 'subopt'
        data_x = program + "_basepairing_x"
        data_y = program + "_basepairing_y"
        mfe_name = program + "_energy"
        composition_name = program + "_composition"

        #Format of .ct file

        #Header: <Total # nt> \t dG = <# mfe> kcal/mol \t <name of sequence>
        #The Rest:
        #<nt num> \t <bp letter> \t <3' neighbor> \t <5' neighbor> \t <# of bp'ing, 0 if none> \t ...
        #<strand-specific nt num> \t <3' neighbor if connected by helix> \t <5' neighbor if connected by helix>

        #Extract the data for the desired complex using complex_ID
        bp_x = self[data_x][complex_ID]
        bp_y = self[data_y][complex_ID]
        mfe = self[mfe_name][complex_ID]

        if program == "mfe" or program == "subopt" or program == "energy":
            composition = self[composition_name]
        elif program == "ordered" or program == "unordered":
            composition = self[composition_name][complex_ID]


        #Determine concatenated sequence of all strands, their beginnings, and ends
        allseq = ""
        strand_begins = []
        strand_ends = []

        #Seemingly, the format of the composition is different for the program complex vs. mfe/subopt
        #for mfe/subopt, the composition is the list of strand ids
        #for complex, it is the number of each strand (in strand id order) in the complex
        #for mfe/subopt, '1 2 2 3' refers to 1 strand of 1, 2 strands of 2, and 1 strand of 3.
        #for complex, '1 2 2 3' refers to 1 strand of 1, 2 strands of 2, 2 strands of 3, and 3 strands of 4'.
        #what a mess.

        if program == "mfe" or program == "subopt" or program == "energy":
            for strand_id in composition:
                strand_begins.append(len(allseq) + 1)
                allseq = allseq + self["sequences"][strand_id-1]
                strand_ends.append(len(allseq))

        else:
            for (num_strands,strand_id) in zip(composition,range(len(composition))):
                for j in range(num_strands):
                    strand_begins.append(len(allseq) + 1)
                    allseq = allseq + self["sequences"][strand_id]
                    strand_ends.append(len(allseq))

        seq_len = len(allseq)

        #print "Seq Len = ", seq_len, "  Composition = ", composition
        #print "Sequence = ", allseq
        #print "Base pairing (x) = ", bp_x
        #print "Base pairing (y) = ", bp_y


        #Create the header
        header = str(seq_len) + "\t" + "dG = " + str(mfe) + " kcal/mol" + "\t" + name + "\n"

        #Open the file
        handle = open(filename,"w")

        #Write the header
        handle.write(header)

        #Write a line for each nt in the secondary structure
        for i in range(1,seq_len+1):


            for (nt,pos) in zip(strand_begins,range(len(strand_begins))):
                if i >= nt:
                    strand_id = pos


            #Determine 3' and 5' neighbor
            #If this is the beginning of a strand, then the 3' neighbor is 0
            #If this is the end of a strand, then the 5' neighbor is 0

            if i in strand_begins:
                nb_5p = 0
            else:
                nb_5p = i - 1

            if i in strand_ends:
                nb_3p = 0
            else:
                nb_3p = i + 1


            if i in bp_x or i in bp_y:
                if i in bp_x: nt_bp = bp_y[bp_x.index(i)]
                if i in bp_y: nt_bp = bp_x[bp_y.index(i)]
            else:
                nt_bp = 0

            #Determine strand-specific counter
            strand_counter = i - strand_begins[strand_id] + 1

            #Determine the 3' and 5' neighbor helical connectivity
            #If the ith nt is connected to its 3', 5' neighbor by a helix, then include it
            #Otherwise, 0
            #Helix connectivity conditions:
            #The 5' or 3' neighbor is connected via a helix iff:
            #a) helix start: i not bp'd, i+1 bp'd, bp_id(i+1) - 1 is bp'd, bp_id(i+1) + 1 is not bp'd
            #b) helix end: i not bp'd, i-1 bp'd, bp_id(i-1) - 1 is not bp'd, bp_id(i-1) + 1 is bp'd
            #c) helix continued: i and bp_id(i)+1 is bp'd, 5' helix connection is bp_id(bp_id(i)+1)
            #d) helix continued: i and bp_id(i)-1 is bp'd, 3' helix connection is bp_id(bp_id(i)-1)
            #Otherwise, zero.

            #Init
            hc_5p = 0
            hc_3p = 0

            if i in bp_x or i in bp_y:  #helix continued condition (c,d)
                if i in bp_x: bp_i = bp_y[bp_x.index(i)]
                if i in bp_y: bp_i = bp_x[bp_y.index(i)]

                if bp_i+1 in bp_x or bp_i+1 in bp_y: #helix condition c
                    if bp_i+1 in bp_x: hc_3p = bp_y[bp_x.index(bp_i+1)]
                    if bp_i+1 in bp_y: hc_3p = bp_x[bp_y.index(bp_i+1)]

                if bp_i-1 in bp_x or bp_i-1 in bp_y: #helix condition d
                    if bp_i-1 in bp_x: hc_5p = bp_y[bp_x.index(bp_i-1)]
                    if bp_i-1 in bp_y: hc_5p = bp_x[bp_y.index(bp_i-1)]

            else: #helix start or end (a,b)

                if i+1 in bp_x or i+1 in bp_y: #Start, condition a
                    if i+1 in bp_x: bp_3p = bp_y[bp_x.index(i+1)]
                    if i+1 in bp_y: bp_3p = bp_x[bp_y.index(i+1)]

                    if bp_3p + 1 not in bp_x and bp_3p + 1 not in bp_y:
                        hc_3p = i + 1

                if i-1 in bp_x or i-1 in bp_y: #End, condition b
                    if i-1 in bp_x: bp_5p = bp_y[bp_x.index(i-1)]

                    if i-1 in bp_y: bp_5p = bp_x[bp_y.index(i-1)]

                    if bp_5p - 1 not in bp_x and bp_5p - 1 not in bp_y:
                        hc_5p = i - 1


            line = str(i) + "\t" + allseq[i-1] + "\t" + str(nb_5p) + "\t" + str(nb_3p) + "\t" + str(nt_bp) + "\t" + str(strand_counter) + "\t" + str(hc_5p) + "\t" + str(hc_3p) + "\n"

            handle.write(line)

        #Close the file. Done.
        handle.close()

if __name__ == "__main__":

    import re

    #sequences = ["AAGATTAACTTAAAAGGAAGGCCCCCCATGCGATCAGCATCAGCACTACGACTACGCGA","acctcctta","ACGTTGGCCTTCC"]
    sequences = ["AAGATTAACTTAAAAGGAAGGCCCCCCATGCGATCAGCATCAGCACTACGACTACGCGA"]

    #Complexes
    #Input: Max number of strands in a complex. Considers all possible combinations of strands, up to max #.
    #'mfe': calculate mfe? 'ordered': consider ordered or unordered complexes?
    #Other options available (see function)

    AddComplexes = []
    test = NuPACK(sequences,"rna1999")
    test.complexes(3,mfe = 1, ordered=1)

    print test

    strand_compositions = test["ordered_composition"]
    num_complexes = len(strand_compositions)
    num_strands = len(sequences)

    for counter in range(num_complexes):
        output = "Complex #" + str(counter+1) + " composition: ("
        for strand_id in strand_compositions[counter][0:num_strands-1]:
            output = output + str(strand_id) + ", "
        output = output + str(strand_compositions[counter][num_strands-1]) + ")"

        output = output + "  dG (RT ln Q): " + str(test["ordered_energy"][counter]) + " kcal/mol"
        output = output + "  # Permutations: " + str(test["ordered_permutations"][counter])
        print output
        test.export_PDF(counter, name = "Complex #" + str(counter+1), filename = "Complex_" + str(counter) + ".pdf", program = "ordered")

    #Mfe
    #Input: Number of each strand in complex.
    #Options include RNA/DNA model, temperature, dangles, etc. (See function).
    #Example: If there are 3 unique strands (1, 2, 3), then [1, 2, 3] is one of each strand and [1, 1, 2, 2, 3, 3] is two of each strand.

    #test.mfe([1, 2], dangles = "all")
    #num_complexes = test["mfe_NumStructs"]  #Number of degenerate complexes (same energy)
    #dG_mfe = test["mfe_energy"]
    #print "There are ", num_complexes, " configuration(s) with a minimum free energy of ", dG_mfe, " kcal/mol."


SyntaxError: ignored

In [ ]:
#@title the RBS calculater

#Given an mRNA sequence, this Python class predicts the dG_total and translation initiation rate.
#This file is part of the Ribosome Binding Site Calculator.
#along with Ribosome Binding Site Calculator.  If not, see <http://www.gnu.org/licenses/>.


from NuPACK import NuPACK
import re
import math

class CalcError(Exception):
    """Base class for exceptions in this module."""

    def __init__(self, value):
        self.value = value

    def __str__(self):
        return repr(self.value)

class RBS_Calculator(NuPACK):

    #From experimental characterization of ALL Predicted RBSs as of 1/20/08
    RT_eff = 2.222
    logK =  7.824
    K = 2500.0

    #Global parameters -- constants
    infinity = 1e12 #For all practical purposes, here.
    RNA_model = "rna1999"
    start_codon_energies = {"ATG":-1.194, "AUG": -1.194, "GTG": -0.0748, "GUG": -0.0748, "TTG":-0.0435, "UUG": -0.0435, "CTG": -0.03406, "CUG":-0.03406} #hybridization to CAT
    auto_dangles = True
    dangles_default = "all"
    temp = 37.0
    optimal_spacing = 5 #aligned spacing

    dG_spacing_constant_push = [12.2, 2.5, 2.0, 3.0]
    dG_spacing_constant_pull = [0.048, 0.24, 0.0]
    cutoff = 35 #number of nt +- start codon considering for folding
    standby_site_length = 4 #Number of nt before SD sequence that must be unpaired for ribosome binding
    energy_cutoff = 3.0
    start_codons = ["ATG", "AUG", "GTG", "GUG","TTG","UUG"] #substituted U for T in actual calcs. Ignores CTG/CUG
    rRNA = "acctcctta" #These are the last 9 nt (3' end) of the 16S rRNA in E. coli

    footprint = 1000 #Footprint of the 30S complex that prevents formation of secondary structures downstream of the start codon. Here, we assume that the entire post-start RNA sequence does not form secondary structures once the 30S complex has bound.

    def __init__(self, mRNA, start_range, name = "Unnamed mRNA", verbose = False):
        """Initializes the RBS Calculator class with the mRNA sequence and the range of start codon positions considered."""

        #NuPACK.__init__(self,sequences,self.RNA_model)

        exp = re.compile('[ATGCU]',re.IGNORECASE)
        if exp.match(mRNA) == None:
            raise ValueError("Invalid letters found in sequence ""%s"". Only ATGCU accepted." % mRNA)

        if start_range[0] < 0: start_range[0] = 0
        if start_range[1] > len(mRNA): start_range[1] = len(mRNA)

        self.name = name
        self.mRNA_input = mRNA.upper()
        self.rRNA_len = len(self.rRNA)
        self.mRNA_len = len(self.mRNA_input)
        self.total_sequence_length = len(mRNA) + len(self.rRNA)
        self.dG_rRNA = self.calc_dG_rRNA()
        self.run = 0
        self.start_range = start_range
        self.verbose = verbose

    def find_min(self,input_list):
        """Finds the minimum of a list of numbers."""

        min_item = self.infinity
        min_index = 0

        for i, item in enumerate(input_list):
            if item < min_item:
                min_item = item
                min_index = i

        return (min_item,min_index)


    def find_start_codons(self,sequence):
        """Finds all start codons in an mRNA sequence. Creates a list."""

        self.start_position_list = []
        self.start_codon_list = []

        seq_len = len(sequence)
        end = min(self.start_range[1],seq_len-2)
        begin = min(self.start_range[0],end)

        for i in range(begin,end+1):
            codon = sequence[i:i+3]
            if codon.upper() in self.start_codons:
                self.start_position_list.append(i)
                self.start_codon_list.append(codon)
                yield (i,codon)
            else:
                pass

    def calc_aligned_spacing(self,mRNA,start_pos,bp_x,bp_y):
        """Calculates the aligned spacing between the 16S rRNA binding site and the start codon."""

        #rRNA is the concatenated at the end of the sequence in 5' to 3' direction
        #first: identify the farthest 3' nt in the rRNA that binds to the mRNA and return its mRNA base pairer

        Ok = False
        seq_len = len(mRNA) + self.rRNA_len
        for (rRNA_nt) in range(seq_len,seq_len - self.rRNA_len,-1):

            if rRNA_nt in bp_y:
                rRNA_pos = bp_y.index(rRNA_nt)
                if bp_x[rRNA_pos] < start_pos:
                    Ok = True
                    farthest_3_prime_rRNA = rRNA_nt - len(mRNA)

                    mRNA_nt = bp_x[rRNA_pos]
                    distance_to_start = start_pos - mRNA_nt + 1 #start_pos is counting starting from 0 (python)

                    break
                else:
                    break

        if Ok:
            aligned_spacing = distance_to_start - farthest_3_prime_rRNA
        else:
            aligned_spacing = self.infinity

        return aligned_spacing

    def calc_dG_spacing(self, aligned_spacing):
        """Calculates the dG_spacing according to the value of the aligned spacing. This relationship was determined through experiments."""

        if (aligned_spacing < self.optimal_spacing):
            ds = aligned_spacing - self.optimal_spacing

            dG_spacing_penalty = self.dG_spacing_constant_push[0] / (1.0 + math.exp(self.dG_spacing_constant_push[1]*(ds + self.dG_spacing_constant_push[2] )))**self.dG_spacing_constant_push[3]

        else:
            ds = aligned_spacing - self.optimal_spacing
            dG_spacing_penalty = self.dG_spacing_constant_pull[0] * ds * ds + self.dG_spacing_constant_pull[1] * ds + self.dG_spacing_constant_pull[2]

        return dG_spacing_penalty

    def calc_dG_mRNA_rRNA(self,start_pos):
        """Calculates the dG_mRNA_rRNA from the mRNA and rRNA sequence. Considers all feasible 16S rRNA binding sites and includes the effects of non-optimal spacing."""

        begin = max(0,start_pos-self.cutoff)
        mRNA_len = min(len(self.mRNA_input),start_pos+self.cutoff)
        start_pos_in_subsequence = min(start_pos, self.cutoff)
        startpos_to_end_len = mRNA_len - start_pos_in_subsequence - begin

        #1. identify a list of rRNA-binding sites. Binding sites are hybridizations between the mRNA and rRNA and can include mismatches, bulges, etc. Intra-molecular folding is also allowed within the mRNA. The subopt program is used to generate a list of optimal & suboptimal binding sites.
        #Constraints: the entire rRNA-binding site must be upstream of the start codon
        mRNA = self.mRNA_input[begin:start_pos]

        if len(mRNA) == 0:
            raise CalcError("Warning: There is a leaderless start codon, which is being ignored.")

        #print "After exception"

        fold = NuPACK([mRNA,self.rRNA],material = self.RNA_model)
        fold.subopt([1, 2],self.energy_cutoff,dangles = self.dangles, Temp = self.temp)

        if len(fold["subopt_basepairing_x"]) == 0:
            raise CalcError("Warning: The 16S rRNA has no predicted binding site. Start codon is considered as leaderless and ignored.")

        #2. Calculate dG_spacing for each 16S rRNA binding site

        #Calculate the aligned spacing for each binding site in the list
        aligned_spacing = []
        for (bp_x, bp_y) in zip(fold["subopt_basepairing_x"], fold["subopt_basepairing_y"]):
            aligned_spacing.append(self.calc_aligned_spacing(mRNA, start_pos_in_subsequence, bp_x,bp_y))

        dG_spacing_list = []
        dG_mRNA_rRNA = []
        dG_mRNA_rRNA_withspacing = []

        #Calculate dG_spacing using aligned spacing value. Add it to dG_mRNA_rRNA.
        for (counter) in range(len(fold["subopt_basepairing_x"])):

            dG_mRNA_rRNA.append(fold["subopt_energy"][counter])
            val = self.calc_dG_spacing(aligned_spacing[counter])
            dG_spacing_list.append(val)
            dG_mRNA_rRNA_withspacing.append(val + fold["subopt_energy"][counter])

        #3. Find 16S rRNA binding site that minimizes dG_spacing+dG_mRNA_rRNA.
        [dG_mRNA_rRNA_folding, index] = self.find_min(dG_mRNA_rRNA_withspacing)
        dG_spacing_final = dG_spacing_list[index]

        dG_mRNA_rRNA_nospacing = dG_mRNA_rRNA[index]

        #Check: Is the dG spacing large compared to the energy gap? If so, this means the list of suboptimal 16S rRNA binding sites generated by subopt is too short.
        if dG_spacing_final > self.energy_cutoff:
            if self.verbose: print "Warning: The spacing penalty is greater than the energy gap. dG (spacing) = ", dG_spacing_final


        #4. Identify the 5' and 3' ends of the identified 16S rRNA binding site. Create a base pair list.

        most_5p_mRNA = self.infinity
        most_3p_mRNA = -self.infinity

        #Generate a list of rRNA-mRNA base paired nucleotides
        bp_x_target = []
        bp_y_target = []

        bp_x = fold["subopt_basepairing_x"][index]
        bp_y = fold["subopt_basepairing_y"][index]
        for (nt_x, nt_y) in zip(bp_x, bp_y):
            if nt_y > len(mRNA): #nt is rRNA
                most_5p_mRNA = min(most_5p_mRNA, bp_x[bp_y.index(nt_y)])
                most_3p_mRNA = max(most_3p_mRNA, bp_x[bp_y.index(nt_y)])
                bp_x_target.append(nt_x)
                bp_y_target.append(nt_y)

        #The rRNA-binding site is between the nucleotides at positions most_5p_mRNA and most_3p_mRNA
        #Now, fold the pre-sequence, rRNA-binding-sequence and post-sequence separately. Take their base pairings and combine them together. Calculate the total energy. For secondary structures, this splitting operation is allowed.

        #We postulate that not all of the post-sequence can form secondary structures. Once the 30S complex binds to the mRNA, it prevents the formation of secondary structures that are mutually exclusive with ribosome binding. We define self.footprint to be the length of the 30S complex footprint. Here, we assume that the entire mRNA sequence downstream of the 16S rRNA binding site can not form secondary structures.

        mRNA_pre = self.mRNA_input[begin:begin+most_5p_mRNA-1]
        post_window_end = mRNA_len + 1
        post_window_begin = min(start_pos + self.footprint,post_window_end) #Footprint
        post_window_end = mRNA_len + 1
        mRNA_post = self.mRNA_input[post_window_begin:post_window_end]

        mRNA_pre_len = len(mRNA_pre)
        mRNA_post_len = len(mRNA_post)
        mRNA_rRNA_binding_len = most_3p_mRNA - most_5p_mRNA + 1
        total_folded_len = mRNA_pre_len + mRNA_post_len + mRNA_rRNA_binding_len

        total_bp_x = []
        total_bp_y = []

        #Calculate pre-sequence folding
        if len(mRNA_pre) > 0:
            fold_pre = NuPACK([mRNA_pre], material = self.RNA_model)
            fold_pre.mfe([1], dangles = self.dangles, Temp = self.temp)
            bp_x_pre = fold_pre["mfe_basepairing_x"][0]
            bp_y_pre = fold_pre["mfe_basepairing_y"][0]

        else:
            bp_x_pre = []
            bp_y_pre = []

        #Add pre-sequence base pairings to total base pairings
        offset = 0 #Begins at 0
        for (nt_x, nt_y) in zip(bp_x_pre, bp_y_pre):
            total_bp_x.append(nt_x + offset)
            total_bp_y.append(nt_y + offset)

        #Add rRNA-binding site base pairings to total base pairings
        offset = 0 #Begins at zero
        if startpos_to_end_len < self.cutoff:
            rRNA_offset = startpos_to_end_len
        else:
            rRNA_offset = startpos_to_end_len

        for (nt_x, nt_y) in zip(bp_x_target, bp_y_target):
            total_bp_x.append(nt_x + offset)
            total_bp_y.append(nt_y + rRNA_offset)

        #Calculate post-sequence folding
        if len(mRNA_post) > 0:
            fold_post = NuPACK([mRNA_post], material = self.RNA_model)
            fold_post.mfe([1], dangles = self.dangles, Temp = self.temp)
            bp_x_post = fold_post["mfe_basepairing_x"][0]
            bp_y_post = fold_post["mfe_basepairing_y"][0]
        else:
            bp_x_post = []
            bp_y_post = []

        offset = post_window_begin - begin
        for (nt_x, nt_y) in zip(bp_x_post, bp_y_post):
            total_bp_x.append(nt_x + offset)
            total_bp_y.append(nt_y + offset)

        mRNA = self.mRNA_input[begin:mRNA_len]
        fold = NuPACK([mRNA, self.rRNA], material = self.RNA_model)

        total_energy = fold.energy([1, 2], total_bp_x, total_bp_y, Temp = self.temp, dangles = self.dangles)

        energy_nowindows = dG_mRNA_rRNA_nospacing
        total_energy_withspacing = total_energy + dG_spacing_final

        structure = fold
        structure["program"] = "subopt"
        structure["mRNA"] = mRNA
        structure["MinStructureID"] = 0
        structure["dG_mRNA_rRNA"] = total_energy
        structure["dG_mRNA_rRNA_withspacing"] = total_energy_withspacing
        structure["dG_spacing"] = dG_spacing_final
        structure["subopt_energy"] = [total_energy_withspacing]
        structure["subopt_basepairing_x"] = [total_bp_x]
        structure["subopt_basepairing_y"] = [total_bp_y]
        structure["subopt_composition"] = [1, 2]
        structure["bp_x"] = total_bp_x
        structure["bp_y"] = total_bp_y

        return (total_energy_withspacing, structure)

    def calc_dG_standby_site(self,structure_old, rRNA_binding = True):
        """Calculates the dG_standby given the structure of the mRNA:rRNA complex"""

        #To calculate the mfe structure while disallowing base pairing at the standby site, we split the folded mRNA sequence into three parts: (i) a pre-sequence (before the standby site) that can fold; (ii) the standby site, which can not fold; (iii) the 16S rRNA binding site and downstream sequence, which has been previously folded.

        import copy
        structure = copy.deepcopy(structure_old)
        mRNA = structure["mRNA"]
        bp_x = structure["bp_x"]
        bp_y = structure["bp_y"]
        energy_before = structure["dG_mRNA_rRNA"] #without spacing effects

        #Identify the most 5p mRNA nt that is bound to rRNA
        for (nt_x, nt_y) in zip(bp_x, bp_y):
            if nt_x <= len(mRNA) and nt_y > len(mRNA): #nt_x is mRNA, nt_y is rRNA, they are bound.
                most_5p_mRNA = nt_x #starts counting from 0
                break

        #Extract the base pairings that are 3' of the most_5p_mRNA base pairing
        bp_x_3p = []
        bp_y_3p = []
        for (nt_x, nt_y) in zip(bp_x, bp_y):
            if nt_x >= most_5p_mRNA:
                bp_x_3p.append(nt_x)
                bp_y_3p.append(nt_y)

        #Create the mRNA subsequence
        mRNA_subsequence = mRNA[0:max(0,most_5p_mRNA - self.standby_site_length - 1)]
        standby_site = mRNA[most_5p_mRNA - self.standby_site_length - 1:most_5p_mRNA]

        #Fold it and extract the base pairings
        if (len(mRNA_subsequence)) > 0:
            fold = NuPACK([mRNA_subsequence], material = self.RNA_model)
            fold.mfe([1], dangles = self.dangles, Temp = self.temp)
            energy_after_5p = fold["mfe_energy"][0]
            bp_x_5p = fold["mfe_basepairing_x"][0]   #[0] added 12/13/07
            bp_y_5p = fold["mfe_basepairing_y"][0]
        else:
             bp_x_5p = []
             bp_y_5p = []
             energy_after_5p = 0.0

        #Put the sets of base pairings together
        bp_x_after = []
        bp_y_after = []
        for (nt_x, nt_y) in zip(bp_x_5p, bp_y_5p):
            bp_x_after.append(nt_x)
            bp_y_after.append(nt_y)

        for (nt_x, nt_y) in zip(bp_x_3p, bp_y_3p):
            bp_x_after.append(nt_x)
            bp_y_after.append(nt_y)

        #Calculate its energy
        fold = NuPACK([mRNA, self.rRNA], material = self.RNA_model)
        energy_after = fold.energy([1, 2], bp_x_after, bp_y_after, dangles = self.dangles, Temp = self.temp)

        dG_standby_site = energy_before - energy_after

        if (dG_standby_site > 0.0): dG_standby_site = 0.0

        index = structure["MinStructureID"]
        structure["bp_x"] = bp_x_after
        structure["bp_y"] = bp_y_after
        structure["subopt_basepairing_x"][index] = bp_x_after
        structure["subopt_basepairing_y"][index] = bp_y_after
        structure["subopt_energy"][index] = energy_after
        structure["dG_mRNA_rRNA_corrected"] = energy_after

        return (dG_standby_site, structure)

    def calc_dG_mRNA(self,start_pos):
        """Calculates the dG_mRNA given the mRNA sequence."""

        mRNA = self.mRNA_input[max(0,start_pos-self.cutoff):min(len(self.mRNA_input),start_pos+self.cutoff)]
        fold = NuPACK([mRNA],self.RNA_model)
        fold.mfe([1], Temp = self.temp, dangles = self.dangles)

        structure = fold
        structure["mRNA"] = mRNA
        structure["bp_x"] = fold["mfe_basepairing_x"][0]
        structure["bp_y"] = fold["mfe_basepairing_y"][0]
        structure["dG_mRNA"] = fold["mfe_energy"][0]
        structure["MinStructureID"] = 0

        dG_mRNA_folding = fold["mfe_energy"][0]

        return (dG_mRNA_folding, structure)

    def calc_dG_rRNA(self):
        """Calculates the dG of folding for the last 9 nt of the 16S rRNA. Not used in the free energy model."""
        fold = NuPACK([self.rRNA],self.RNA_model)
        fold.mfe([1], Temp = self.temp, dangles = "all")
        dG_rRNA_folding = fold["mfe_energy"][0]
        return dG_rRNA_folding

    def calc_dG_SDopen(self, mRNA_structure, mRNA_rRNA_structure):
        """Calculate the dG required to unfold the nucleotides in the 16S rRNA binding site."""

        mRNA = mRNA_structure["mRNA"]
        program = mRNA_structure["program"]
        index = mRNA_structure["MinStructureID"]
        dG_mRNA = mRNA_structure[program + "_energy"][index]

        index = mRNA_rRNA_structure["MinStructureID"]
        bp_x_1 = mRNA_rRNA_structure["subopt_basepairing_x"][index][:]
        bp_y_1 = mRNA_rRNA_structure["subopt_basepairing_y"][index][:]

        most_5p_mRNA = self.infinity
        most_3p_mRNA = -self.infinity

        for (nt_x, nt_y) in zip(bp_x_1, bp_y_1):
            if nt_y > len(mRNA): #nt is rRNA
                most_5p_mRNA = min(most_5p_mRNA, bp_x_1[bp_y_1.index(nt_y)])
                most_3p_mRNA = max(most_3p_mRNA, bp_x_1[bp_y_1.index(nt_y)])

        pre_mRNA = mRNA[0:most_5p_mRNA]
        post_mRNA = mRNA[most_3p_mRNA+1:len(mRNA)+1]

        pre_fold = NuPACK([pre_mRNA],material = self.RNA_model)
        pre_fold.mfe([1],dangles = self.dangles, Temp = self.temp)
        dG_pre = pre_fold["mfe_energy"][0]

        post_fold = NuPACK([post_mRNA],material = self.RNA_model)
        post_fold.mfe([1],dangles = self.dangles, Temp = self.temp)
        dG_post = post_fold["mfe_energy"][0]

        energy = dG_pre + dG_post

        ddG_mRNA = energy - dG_mRNA #positive if work is required to unfold SD sequence
        return ddG_mRNA

    def calc_kinetic_score(self, structure = None, mRNA_in = None, bp_x_in = None, bp_y_in = None):
        """Calculate a "kinetic score", a heuristic measure of the maximum time required for the mRNA secondary structure to form. This is related to the RNA polymer model by David et. al. This heuristic should not be used in any way to quantify the folding kinetics of an mRNA sequence because it completely ignores cooperative RNA folding mechanisms, such as zipping or strand displacement. Here, we use it to eliminate mRNA sequences that MAY fold slowly."""

        if not (structure is None):
            program = structure["program"]
            mRNA = structure["mRNA"]
            index = structure["MinStructureID"]
            bp_x = structure[program + "_basepairing_x"][index]
            bp_y = structure[program + "_basepairing_y"][index]

        if not (bp_x_in is None) and not (bp_y_in is None) and not (mRNA_in is None):
            mRNA = mRNA_in[:]
            bp_x = bp_x_in[:]
            bp_y = bp_y_in[:]

        largest_range_helix = 0
        for (nt_x, nt_y) in zip(bp_x, bp_y):
            if nt_x <= len(mRNA) and nt_y <= len(mRNA):
                val = nt_y - nt_x
                largest_range_helix = max(val, largest_range_helix)

        kinetic_score = float(largest_range_helix) / float(len(mRNA))
        if float(largest_range_helix) > 0:
            min_bp_prob = float(largest_range_helix)**(-1.44) #From David et. al.
        else:
            min_bp_prob = 1.0

        return (kinetic_score, min_bp_prob)

    def calc_most_5p_mRNA(self,structure_old):
        """Calculates the most 5' nucleotide in the 16S rRNA binding site."""

        import copy
        structure = copy.deepcopy(structure_old)
        mRNA = structure["mRNA"]
        bp_x = structure["bp_x"]
        bp_y = structure["bp_y"]

        #Identify the most 5p mRNA nt that is bound to rRNA
        for (nt_x, nt_y) in zip(bp_x, bp_y):
            if nt_x <= len(mRNA) and nt_y > len(mRNA): #nt_x is mRNA, nt_y is rRNA, they are bound.
                most_5p_mRNA = nt_x
                break

        return most_5p_mRNA

    def calc_longest_helix(self,structure):
        """Calculate the longest helical structure (longest contiguous list of base pairings) in the secondary structure"""

        mRNA = structure["mRNA"]
        bp_x = structure["bp_x"]
        bp_y = structure["bp_y"]

        longest_helix = 0
        helix_length = 1

        for (nt_x, nt_y) in zip(bp_x, bp_y):
            if (bp_x.count(nt_x+1) > 0 and bp_y.count(nt_y-1) > 0):
                helix_length += 1
            else:
                longest_helix = max(longest_helix, helix_length)
                helix_length = 1

        return longest_helix

    def calc_longest_loop_bulge(self,structure,output_start_end=False,InRBSOnly=False,RBS=None):
        """Calculate the longest helical loop and bulge structure (longest contiguous list of un-base paired nucleotides starting and ending with a helix (loop -> same helix, bulge -> different helix) in the secondary structure"""

        mRNA = structure["mRNA"]
        bp_x = structure["bp_x"]
        bp_y = structure["bp_y"]

        loop_length = 0
        begin_helix = 1

        bulge_loop_list = []
        helical_loop_list = []

        if output_start_end:
            bulge_loop_start_end = []
            helical_loop_start_end = []

        if InRBSOnly and RBS is not None:
            RBS_begin = mRNA.find(RBS)
            RBS_end = RBS_begin + len(RBS)
            nucleotide_range = range(RBS_begin,RBS_end+1)

        else:
            nucleotide_range = range(1,len(mRNA)+1)

        #Find loops. Find bulges.
        for n in nucleotide_range:
            if bp_x.count(n) == 0 and bp_y.count(n) == 0: #nth nucleotide is not base-paired.

                #Determine if nearest neighbor nucleotides are base-paired
                (x1,x2,y1,y2) = (bp_x.count(n-1), bp_x.count(n+1), bp_y.count(n-1), bp_y.count(n+1))

                #print "#", n, (x1,x2,y1,y2)

                #middle unpaired nt
                if (x1,x2,y1,y2) == (0,0,0,0):
                    loop_length += 1

                #single mismatch -- loop
                elif (x1,x2,y1,y2) == (1,0,0,1) or (x1,x2,y1,y2) == (0,1,1,0):
                    loop_length += 1
                    begin_helix = n - 1
                    end_helix = n + 1

                #single mismatch -- bulge
                elif (x1,x2,y1,y2) == (1,1,0,0) or (x1,x2,y1,y2) == (0,0,1,1):
                    loop_length += 1
                    begin_helix = n - 1
                    end_helix = n + 1

                #starting unpaired nt
                elif (x1,x2,y1,y2) == (1,0,0,0) or (x1,x2,y1,y2) == (0,0,1,0):
                    loop_length += 1
                    begin_helix = n - 1

                #ending unpaired nt
                elif (x1,x2,y1,y2) == (0,1,0,0) or (x1,x2,y1,y2) == (0,0,0,1):
                    loop_length += 1
                    end_helix = n + 1

                #1,0,1,0 is impossible w/o psuedoknots
                #0,1,0,1 is impossible w/o psuedoknots
                #Also, all binary combinations with 3 or 4 true are impossible (n-1 or n+1 can not be in both bp_x and bp_y)


            elif loop_length > 0:
                #Bulge or loop?
                #loop

                #print "begin = ", begin_helix
                #print "end = ", end_helix

                if ( bp_x.count(begin_helix) > 0 and bp_y.count(end_helix) > 0 and bp_x.index(begin_helix) == bp_y.index(end_helix) ):
                    helical_loop_list.append(loop_length)
                    loop_length = 0

                    if output_start_end:
                        #Also return the starting and ending positions of each loop/bulge
                        helical_loop_start_end.append((begin_helix,end_helix))

                else:

                    bp_end = 0
                    bp_begin = 0

                    if (bp_x.count(end_helix) > 0): bp_begin = bp_y[bp_x.index(end_helix)]
                    if (bp_y.count(end_helix) > 0): bp_end = bp_x[bp_y.index(end_helix)]

                    if (bp_x.count(begin_helix) > 0): bp_end = bp_y[bp_x.index(begin_helix)]
                    if (bp_y.count(begin_helix) > 0): bp_begin = bp_x[bp_y.index(begin_helix)]

                    if bp_end > bp_begin:
                        bulge_loop_list.append(loop_length)
                        loop_length = 0

                        if output_start_end:
                            #Also return the starting and ending positions of each loop/bulge
                            bulge_loop_start_end.append((begin_helix,end_helix))

                    else:
                        loop_length = 0
        if output_start_end:
            return (helical_loop_list, bulge_loop_list, helical_loop_start_end, bulge_loop_start_end)
        else:
            return (helical_loop_list, bulge_loop_list)

    def cpu_time(self):
        import resource
        return resource.getrusage(resource.RUSAGE_SELF)[0]

    def combsort(self, input_list):
        """Sorts the input_list in increasing order (minimum first) according to the comb sort algorithm from Wikipedia. Outputs the corresponding sorted index. """

        index = range(len(input_list))

        #Implementation of comb sort (from Wikipedia)
        shrink_factor = 1.24733095

        #Init
        gap = len(input_list)
        swaps = False

        while not (gap <= 1 and not swaps):

            #update the gap value for a next comb
            if gap > 1:
                gap = int(gap / shrink_factor)
                if gap == 10 or gap == 9:
                    gap = 11

            i = 0
            swaps = False #see bubblesort for an explanation

            #a single "comb" over the input list
            while gap + i < len(input_list):
                val = input_list[i]
                if val > input_list[i+gap]:
                    #Swap values -- verify that lists are being altered correctly
                    input_list[i] = input_list[gap + i]
                    input_list[gap + i] = val

                    #Swap indices of index list accordingly
                    temp = index[i]
                    index[i] = index[gap + i]
                    index[gap + i] = temp

                    swaps = True

                i += 1

        return index

    def calc_dG(self):
        """Calculates each dG term in the free energy model and sums them together to create dG_total"""

        start = self.cpu_time()

        #Initialization of data structures
        self.start_pos_list = []
        self.dG_total_list = []
        self.dG_mRNA_list = []
        self.dG_mRNA_rRNA_list = []
        self.fold_x_list = []
        self.fold_y_list = []
        self.dG_start_energy_list = []
        self.dG_spacing_list = []
        self.mRNA_structure_list = []
        self.mRNA_rRNA_uncorrected_structure_list = []
        self.mRNA_rRNA_corrected_structure_list = []
        self.dG_standby_site_list = []
        self.dG_spacer_site_list = []
        self.kinetic_score_list = []
        self.min_bp_prob_list = []
        self.longest_helix_list = []
        self.three_state_indicator_list = []
        self.helical_loop_list_list = []
        self.bulge_loop_list_list = []

        self.dS1_list = []
        self.dS2_list = []
        self.most_5p_mRNA_list = []
        self.Expression_list = []

        for (start_pos, codon) in self.find_start_codons(self.mRNA_input):

            try:

                #print "Top of calc_dG here"

                #Set dangles based on length between 5' end of mRNA and start codon
                if self.auto_dangles:

                    if start_pos > self.cutoff:
                        self.dangles = "none"

                    else:
                        self.dangles = "all"

                else:
                    self.dangles = self.dangles_default
                    #print "Auto Dangles set to ", self.dangles

                #Start codon energy
                dG_start_codon = self.start_codon_energies[codon]

                #Energy of mRNA folding
                [dG_mRNA,mRNA_structure] = self.calc_dG_mRNA(start_pos)

                #Energy of mRNA:rRNA hybridization & folding
                [dG_mRNA_rRNA_withspacing,mRNA_rRNA_structure] = self.calc_dG_mRNA_rRNA(start_pos)

                dG_mRNA_rRNA_nospacing = mRNA_rRNA_structure["dG_mRNA_rRNA"]

                #Standby site correction:
                [dG_standby_site, corrected_structure] = self.calc_dG_standby_site(mRNA_rRNA_structure, rRNA_binding = True)

                #Total energy is mRNA:rRNA + start - rRNA - mRNA - standby_site
                dG_total = dG_mRNA_rRNA_withspacing + dG_start_codon - dG_mRNA - dG_standby_site

                #Calculate 'kinetic score': directly related to probability of base pair formation
                (kinetic_score,min_bp_prob) = self.calc_kinetic_score(mRNA_structure)

                #Calculate dG to open SD sequence
                ddG_SD_open = self.calc_dG_SDopen(mRNA_structure, mRNA_rRNA_structure)


                self.mRNA_structure_list.append(mRNA_structure)
                self.mRNA_rRNA_uncorrected_structure_list.append(mRNA_rRNA_structure)
                self.mRNA_rRNA_corrected_structure_list.append(corrected_structure)
                self.most_5p_mRNA_list.append(self.calc_most_5p_mRNA(mRNA_rRNA_structure))

                (helical_loop_list, bulge_loop_list) = self.calc_longest_loop_bulge(structure=mRNA_structure)
                self.longest_helix_list.append(self.calc_longest_helix(structure=mRNA_structure))

                self.dG_start_energy_list.append(dG_start_codon)
                self.dG_mRNA_list.append(dG_mRNA)
                self.dG_mRNA_rRNA_list.append(dG_mRNA_rRNA_nospacing)
                self.dG_spacing_list.append(mRNA_rRNA_structure["dG_spacing"])
                self.dG_standby_site_list.append(dG_standby_site)
                self.dG_total_list.append(dG_total)

                self.helical_loop_list_list.append(helical_loop_list)
                self.bulge_loop_list_list.append(bulge_loop_list)

                self.min_bp_prob_list.append(min_bp_prob)
                self.kinetic_score_list.append(kinetic_score)
                self.three_state_indicator_list.append(ddG_SD_open)

                #Start positions
                self.start_pos_list.append(start_pos)

                #Expression levels
                self.Expression_list.append(self.calc_expression_level(dG_total))

                #For exporting the relevant structure to a PDF
                #index = mRNA_rRNA_structure["MinStructureID"]
                #mRNA_rRNA_structure.export_PDF(index, name = self.name + ": Before standby site", filename =  self.name + "_Before_Standby_rRNA.pdf", program = "subopt")

                #index = corrected_structure["MinStructureID"]
                #corrected_structure.export_PDF(index, name = self.name + ": After standby site", filename =  self.name + "_After_Standby_rRNA.pdf", program = "subopt")

            except CalcError, msg:
                print msg
                self.mRNA_structure_list.append([])
                self.mRNA_rRNA_uncorrected_structure_list.append([])
                self.mRNA_rRNA_corrected_structure_list.append([])

                self.most_5p_mRNA_list.append(self.infinity)
                self.longest_helix_list.append(self.infinity)

                self.dG_start_energy_list.append(self.infinity)
                self.dG_mRNA_list.append(self.infinity)
                self.dG_mRNA_rRNA_list.append(self.infinity)
                self.dG_spacing_list.append(self.infinity)
                self.dG_standby_site_list.append(self.infinity)
                self.dG_total_list.append(self.infinity)

                self.helical_loop_list_list.append(self.infinity)
                self.bulge_loop_list_list.append(self.infinity)

                self.kinetic_score_list.append(self.infinity)
                self.three_state_indicator_list.append(self.infinity)

        self.run = 1

        end = self.cpu_time()
        self.run_time = end - start

    def calc_expression_level(self, dG):

        import math
        return RBS_Calculator.K * math.exp(-dG / RBS_Calculator.RT_eff)

    def print_dG(self,max_dG = 1e12,brief = 0, return_string = False, print_expression = False):
        '''Print out useful information about the mRNA sequence'''

        import math

        print_string = ""
        if self.run == 1:

            print "mRNA Sequence: ", self.name
            if return_string: print_string = print_string + "mRNA Sequence: " + self.name + "\n"

            if len(self.start_position_list) == 0:
                print "No start codons found in input mRNA sequence"
                print "----------------------------------------------------------------------------------------"
                if return_string: print_string = print_string + "No start codons found in input mRNA sequence" + "\n" + "----------------------------------------------------------------------------------------" + "\n"
            elif len( [dG_total for dG_total in self.dG_total_list if dG_total < max_dG] ) == 0:
                print "No RBSs found with dG <", str(max_dG), "."
                print "----------------------------------------------------------------------------------------" + "\n"
                if return_string: print_string = print_string + "No RBSs found with dG <" + str(max_dG) + "." + "\n" + "----------------------------------------------------------------------------------------" + "\n"

            else:

                if print_expression:
                    Headers = ("Start", "(pos)", "Expression level","Kinetic score")
                    format = "%4s %4s %15s %15s"
                else:
                    Headers = ("Start","(pos)", "dG total", "dG (rRNA:mRNA)", "dG (mRNA)", "dG (spacing)", "dG (standby)", "Kinetic Score")
                    format = "%4s %4s %12s %12s %12s %12s %12s %12s"

                print format % Headers

                if return_string: print_string = print_string + format % Headers + "\n"

                for (start,counter) in zip(self.start_position_list,range(len(self.start_position_list))):

                    dG_total = self.dG_total_list[counter]
                    Expression = RBS_Calculator.K * math.exp(-dG_total / RBS_Calculator.RT_eff)

                    if (dG_total < max_dG):

                        if (dG_total > self.infinity):
                            dG_total = "Inf"
                            Expression = 0.0

                        dG_mRNA = self.dG_mRNA_list[counter]
                        if (dG_mRNA > self.infinity): dG_mRNA = "Inf"

                        dG_mRNA_rRNA = self.dG_mRNA_rRNA_list[counter]
                        if (dG_mRNA_rRNA > self.infinity): dG_mRNA_rRNA = "Inf"

                        dG_spacing = self.dG_spacing_list[counter]
                        if (dG_spacing > self.infinity): dG_spacing = "Inf"

                        dG_standby_site = self.dG_standby_site_list[counter]
                        if (dG_standby_site > self.infinity): dG_standby_site = "Inf"

                        start_codon = self.start_codon_list[counter]

                        dG_rRNA = self.dG_rRNA

                        kinetic_score = self.kinetic_score_list[counter]

                        if print_expression:
                            print format % (start_codon, str(start), str(round(Expression,2)), str(round(kinetic_score,2)))

                        else:
                            print format % (start_codon, str(start), str(dG_total), str(dG_mRNA_rRNA), str(dG_mRNA), str(dG_spacing), str(dG_standby_site), str(round(kinetic_score,2)))

                        if return_string: print_string = print_string + format % (start_codon, str(start), str(dG_total), str(dG_mRNA_rRNA), str(dG_mRNA), str(dG_spacing), str(dG_standby_site), str(round(kinetic_score,2))) + "\n"

                print "----------------------------------------------------------------------------------------"
                if return_string: print_string = print_string + "----------------------------------------------------------------------------------------" + "\n"

                #print "Computation Time: ", str(self.run_time), " seconds."

                if return_string: return print_string
        else:
            raise RuntimeError("The RBS Calculator has not been run yet. Call the 'calc_dG' method.")

    def save_data(self, handle, header = False):

        infinity = 1e6

        if self.run == 1:

            if (header):
                #Print header information --> all the parameters
                parameters = "RNA model: \t%s \nDangles: \t%s \nTemperature: \t%s \nOptimal Spacing \t%s \nSpacing constant (push) \t%s\nSpacing constant (pull) \t%s\nCutoff \t%s\nStandby Site Length \t%s\nEnergy Cutoff \t%s\nrRNA sequence \t%s\n" % (self.RNA_model, str(self.dangles), str(self.temp), str(self.optimal_spacing), str(self.dG_spacing_constant_push), str(self.dG_spacing_constant_pull), str(self.cutoff), str(self.standby_site_length), str(self.energy_cutoff), self.rRNA)

                handle.writelines(parameters)

            #Name, dG total, dG rRNA:mRNA, dG mRNA, dG spacing, dG standby, dG start codon, kinetic score, longest helix, longest loop, start position
            #format = "%10s\t%10s\t%10s\t%10s\t%10s\t%10s\t%10s\t%10s\t%10s\t%10s\t%10s\n"
            format = "%10s\t%10s\t%10s\t%10s\t%10s\t%10s\t%10s\t%10s\t%10s\t%10s\t%10s\t%10s\n"

            for counter in range(len(self.start_position_list)):

                dG_total = self.dG_total_list[counter]
                if (dG_total >= infinity): dG_total = "Inf"

                dG_mRNA = self.dG_mRNA_list[counter]
                if (dG_mRNA >= infinity): dG_mRNA = "Inf"

                dG_mRNA_rRNA = self.dG_mRNA_rRNA_list[counter]
                if (dG_mRNA_rRNA >= infinity): dG_mRNA_rRNA = "Inf"

                dG_spacing = self.dG_spacing_list[counter]
                if (dG_spacing >= infinity): dG_spacing = "Inf"

                dG_standby_site = self.dG_standby_site_list[counter]
                if (dG_standby_site >= infinity): dG_standby_site = "Inf"

                kinetic_score = self.kinetic_score_list[counter]

                longest_helix = self.longest_helix_list[counter]
                if len(self.helical_loop_list_list[counter])>0:
                    longest_loop = max(self.helical_loop_list_list[counter])
                else:
                    longest_loop = 0

                most_5p_mRNA = self.most_5p_mRNA_list[counter]

                three_state = self.three_state_indicator_list[counter]

                handle.writelines(format % (self.name.split(" ")[0],dG_total, dG_mRNA_rRNA,dG_mRNA,dG_spacing,dG_standby_site, self.start_codon_energies[self.start_codon_list[counter]],round(kinetic_score,2), str(longest_helix), str(self.start_position_list[counter]), str(most_5p_mRNA), str(three_state) ))

        else:
            raise RuntimeError("The RBS Calculator has not been run yet. Call the 'calc_dG' method.")

#----------------------------------------------------------------------------------------------------------
#End RBS_Calculator class
#----------------------------------------------------------------------------------------------------------
def calc_dG_from_file(handle, output, verbose = True, parameters = {}):
    from Bio import SeqIO
    from RBS_Calculator import RBS_Calculator

    records = SeqIO.parse(handle,"fasta")
    First = True
    export_PDF = True

    #for i in range(30):
    #    records.next()

    for record in records:

        mRNA = record.seq.tostring().upper()

        #Set any defaults
        start_range = [0, len(mRNA)]
        name = record.description.split(" ")[0]

        #Create instance of RBS Calculator
        test = RBS_Calculator(mRNA, start_range, name)

        #Examine kvars dictionary and pull out any options. Assign them to instanced class.
        for (key,value) in parameters.items():

            if key == "cutoff":
                test.cutoff = value
            elif key == "start_range":
                test.start_range = value
            elif key == "rRNA":
                test.rRNA = value
            elif key == "energy_cutoff":
                test.energy_cutoff = value
            elif key == "standby_site_length":
                test.standby_site_length = value
            elif key == "dangles":
                test.dangles = value
            elif key == "export_PDF":
                export_PDF = value

        test.calc_dG()
        test.print_dG(test.infinity,print_expression=verbose)
        test.save_data(output, First)

        if First:
            First = False

        if export_PDF:
            num_structs = len(test.mRNA_rRNA_uncorrected_structure_list)
            for (structure,counter) in zip(test.mRNA_rRNA_uncorrected_structure_list,range(num_structs)):
                index = structure["MinStructureID"]
                structure.export_PDF(index, name, filename =  name + "_rRNA" + "_" + str(counter) + ".pdf", program = "subopt")

            num_structs = len(test.mRNA_structure_list)
            for (structure,counter) in zip(test.mRNA_structure_list,range(num_structs)):
                structure.export_PDF(0, name, filename = name + "_mRNA" + "_" + str(counter) + ".pdf")

    output.close()

def calc_dG_pre_post_RBS(pre_list,post_list,RBS_list,name_list,output,verbose = True, parameters = {}):

    from RBS_Calculator import RBS_Calculator

    First = True

    for (pre,post,RBS,name) in zip(pre_list,post_list,RBS_list,name_list):

        mRNA = pre + RBS + post

        start_range = [0, len(mRNA)]

        #Create instance of RBS Calculator
        test = RBS_Calculator(mRNA, start_range, name)

        #Examine kvars dictionary and pull out any options. Assign them to instanced class.
        for (key,value) in parameters.items():

            if key == "cutoff":
                test.cutoff = value
            elif key == "start_range":
                test.start_range = value
            elif key == "rRNA":
                test.rRNA = value
            elif key == "energy_cutoff":
                test.energy_cutoff = value
            elif key == "standby_site_length":
                test.standby_sitRBSe_length = value
            elif key == "dangles":
                test.dangles = value
            elif key == "export_PDF":
                export_PDF = value

        test.calc_dG()
        if verbose: test.print_dG(test.infinity)

        test.save_data(output, First)
        if First:
            First = False

    output.close()

if __name__ == "__main__":

    from Bio import SeqIO



    #filename = "/common/RBS_Calculator/DataSets/Amin_RBSs.txt"
    #filename = "/common/RBS_Calculator/DataSets/Forward_Predictions.txt"
    #filename = "/common/RBS_Calculator/DataSets/Context_Tests_RBSs_2nd.txt"
    #filename = "/common/RBS_Calculator/DataSets/DNA20_CDSs.fasta"
    #output_filename = "/common/RBS_Calculator/Output_Forward_Predictions_dangles_all_TTC.txt"
    #output_filename = "/common/RBS_Calculator/Output_Context_Tests_RBSs_test.txt"
    filename = "/common/RBS_Calculator/DataSets/All_Tested_RBSs.txt"
    #filename = "E:\RBS_Calculator\DataSets\All_Tested_RBSs.txt"
    #filename = "/common/RBS_Calculator/DataSets/Testing_RBSs.txt"
    #filename = "/common/RBS_Calculator/DataSets/Karsten_nif_RBSs.txt"
    #filename = "/common/RBS_Calculator/Dan_RBS_silk_sequences.fasta"
    #output_filename = "/common/RBS_Calculator/Output_All_Tested_RBSs_newspacing.txt"
    #output_filename = "/common/RBS_Calculator/Output_DNA20_Designed_RBS_1.txt"
    #output_filename = "/common/RBS_Calculator/Output_Context_Tests_RBSs_3.txt"
    #output_filename = "/common/RBS_Calculator/Output_Karsten_nif_RBSs.txt"
    #output_filename = "/common/RBS_Calculator/Amin_RBSs_Output.txt"

    #filename = "/common/RBS_Calculator/DataSets/Crt_version_RBSs.txt"

    #filename = "/common/RBS_Calculator/DataSets/Output_Crt_natural_RBSs.txt"
    #filename = "/common/RBS_Calculator/DataSets/RBS_Spacing.txt"
    #filename = "/common/RBS_Calculator/DataSets/Ethan_RBSs_all.txt"
    #filename = "/common/RBS_Calculator/DataSets/Dan_sicP.fasta"
    #filename = "/common/RBS_Calculator/DataSets/Natural_prg_org_operon.str"


    #output_filename = "/common/RBS_Calculator/Dan_Output_silk_RBSs.txt"
    #handle = open(filename,"rU")
    #output = open(output_filename,"w")

    #verbose = False

    #pars = {"start_range":[18,71],"pre_window_length":0,"post_window_length":0,"export_PDF":False}
    #calc_dG_from_file(handle, output, verbose,pars)
    #handle.close()
    #output.close()

    window_list = range(10,65)
    for window in window_list:
        print "Cutoff = ", window
        output_filename = "/common/RBS_Calculator/DataSets/cutoff/Output_All_Tested_RBSs_cutoff_" + str(window) + ".txt"

        handle = open(filename,"rU")
        output = open(output_filename,"w")

        verbose = True

        pars = {"start_range":[18,44],"export_PDF":False,"cutoff":window}
        calc_dG_from_file(handle, output, verbose,pars)
        handle.close()
        output.close()

    #pre = "ggggaattgtgagcggataacaattcccctctagaa"
    #RBS = "GAGGAAGTTAGTAAGGAGGTCAGGCGA"

    #pre_list = []
    #RBS_list = []
    #CDS_list = []
    #name_list = []
    ##Get protein coding sequence
    #records = SeqIO.parse(handle,"fasta")
    #for record in records:
        #CDS_list.append(record.seq.tostring().upper())
        #pre_list.append(pre)
        #RBS_list.append(RBS)
        #name_list.append(record.description)

    #calc_dG_pre_post_RBS(pre_list,CDS_list,RBS_list,name_list,output,verbose = True, parameters = pars)


    #name = "Test"
    #mRNA = "TTCTAGAGGGGGGATCTCCCCCCAAAAAATAAGAGGTACACATGACTAAAACTTTCAAAGGCTCAGTATTCCCACTGAG"

    #start_range = [0, len(mRNA)]
    #test = RBS_Calculator(mRNA, start_range, name)
    #test.calc_dG()
    #test.print_dG(test.infinity)

    ##test.post_window_length = 19
    #test.auto_dangles = False
    #test.default_dangles = "all"
    #test.pre_window_length = 1000
    #test.post_window_length = 20
    #test.cutoff = 50
    #test.calc_dG()
    #test.print_dG(test.infinity)


    #if export_PDF:
            #num_structs = len(test.mRNA_rRNA_corrected_structure_list)
            #for (structure,counter) in zip(test.mRNA_rRNA_corrected_structure_list,range(num_structs)):

                #index = structure["MinStructureID"]
                #structure.export_PDF(index, name, filename =  name + "_rRNA" + "_" + str(counter) + ".pdf", program = "energy")

            #for (structure,counter) in zip(test.mRNA_rRNA_uncorrected_structure_list,range(num_structs)):

                #index = structure["MinStructureID"]
                #structure.export_PDF(index, name, filename =  name + "_rRNA_uncorrected" + "_" + str(counter) + ".pdf", program = "energy")

            #num_structs = len(test.mRNA_structure_list)
            #for (structure,counter) in zip(test.mRNA_structure_list,range(num_structs)):
                #structure.export_PDF(0, name, filename = name + "_mRNA" + "_" + str(counter) + ".pdf")


In [ ]:
#@title code for Run_RBS_Calculator.py
#This file is part of the Ribosome Binding Site Calculator.

#along with Ribosome Binding Site Calculator.  If not, see <http://www.gnu.org/licenses/>.

from RBS_Calculator import RBS_Calculator
import sys, math

if __name__ == "__main__":
    #Read command line arguments
    input_args = []
    for arg in sys.argv:
        input_args.append(arg)

    start = 0

    if len(input_args) == 3:
        seq = input_args[1]
        start = input_args[2]
    elif len(input_args) == 2:
        seq = input_args[1]
    else:
        output = "Usage: " + input_args[0] + " [RNA/DNA Sequence] (start position)" + "\n"
        print output

    if start == 0:
        start_range = [0, len(seq)]
    else:
        start_range = [int(start), int(start)+1]

    name = "no name"

    #Create instance of RBS Calculator
    calcObj = RBS_Calculator(seq, start_range, name)
    calcObj.calc_dG()

    dG_total_list = calcObj.dG_total_list[:]
    start_pos_list = calcObj.start_pos_list[:]
    kinetic_score_list = calcObj.kinetic_score_list[:]

    expr_list = []
    for dG in dG_total_list:
        expr_list.append(calcObj.K * math.exp(-dG/calcObj.RT_eff))

    print len(expr_list)
    for (expr,start_pos,ks) in zip(expr_list,start_pos_list,kinetic_score_list):
        print start_pos, expr, ks 

In [ ]:
#@title Code RBS_MC_Design

#This Python class generates a synthetic RBS sequence according to a user's target translation initiation rate, protein coding sequence, and pre-sequence. RBS constraints are not allowed.
#This file is part of the Ribosome Binding Site Calculator.
#The Ribosome Binding Site Calculator is free software: you can redistribute it and/or modify


from RBS_Calculator import RBS_Calculator
import random, math, sets

infinity = 1.0e20

#Constraints settings
max_kinetic_score = 0.50
max_three_state_indicator = 6.0
min_helical_loop = 4
max_helical_loop = 12
Max_RBS_Length = RBS_Calculator.cutoff

#dG ranges
dG_range_high = 25.0
dG_range_low = -18.0

num_rbs_calculations = 0

def dsu_sort(idx, seq):
    """Sorts a list of tuples according to the idx column using a Decorate-Sort-Undecorate method"""

    for i, e in enumerate(seq):
        seq[i] = (e[idx], e)
    seq.sort()
    seq.reverse()
    for i, e in enumerate(seq):
        seq[i] = e[1]
    return seq

def weighted_choice(list_of_tuples):
    """Randomly chooses from a list of choices according to their weighted probabilities."""

    n = random.uniform(0.0, 1.0)
    for item, weight in list_of_tuples:
        if n < weight:
            break
        n = n - weight
    return item

def Run_RBS_Calculator(pre_seq,post_seq,RBS,verbose=True):
    """Short cut function to run the RBS Calculator on a pre_sequence, CDS, and RBS."""

    if vars().has_key('estimator'): del(estimator)

    start_range = [len(pre_seq) + len(RBS) - 2, len(pre_seq) + len(RBS) + 2]

    mRNA = pre_seq.upper() + RBS + post_seq.upper()

    estimator = RBS_Calculator(mRNA, start_range, "")
    estimator.calc_dG()
    if verbose: estimator.print_dG()

    global num_rbs_calculations
    num_rbs_calculations+=1

    return estimator

def Generate_Random_RBS(All_Random = False, Max_length = 20, Pre_length = 5, PchooseSD = 0.5, Core_length = 6, max_nonoptimal_spacing = 5):
    """Generates a random RBS sequence tailored towards the target translation initiation rate."""

    RBS = []
    if All_Random:

        for i in range(Max_length):
            RBS.append(random.choice(["A","T","G","C"]))
        return "".join(RBS)

    for i in range(Pre_length):
        RBS.append(random.choice(["A","T","G","C"]))

    SD = ["T","A","A","G","G","A","G","G","T"]

    #Choose Core_length nucleotides. Choose from the SD sequence with probability PchooseSD
    #Choose from non-SD sequence with probability (1 - PchooseSD) / 3
    #The beginning/end of the Core_length wrt to the SD sequence is uniformly randomly determined

    Core_length = min(len(SD),Core_length) #Can't be greater then SD length
    diff = len(SD) - Core_length
    begin = int(random.random() * diff)

    for i in range(Core_length):
        rand = random.random()
        if rand <= PchooseSD:
            RBS.append(SD[begin+i])
        else:
            choices = ["A","T","G","C"]
            choices.remove(SD[begin+i])
            RBS.append(random.choice(choices))

    optimal_spacing = RBS_Calculator.optimal_spacing
    offset = diff - begin

    spacing = random.choice(range(max(0,offset + optimal_spacing - max_nonoptimal_spacing), offset + optimal_spacing + max_nonoptimal_spacing))

    for i in range(spacing):
        RBS.append(random.choice(["A","T","G","C"]))

    if len(RBS) > Max_length:
         RBS = RBS[len(RBS)-Max_length:len(RBS)+1]

    return "".join(RBS)

def calc_constraints(RBS,estimator):
    """Calculates the sequence constraints. Returns True if one is violated. """

    kinetic_score = estimator.kinetic_score_list[0]
    three_state_indicator = estimator.three_state_indicator_list[0]
    #(helical_loop_list,bulge_loop_list,helical_start_ends,bulge_start_ends) = estimator.calc_longest_loop_bulge(estimator,True,True,RBS)

    #print "KS = ", kinetic_score
    #print "3-state = ", three_state_indicator
    #print "max/min helical = ", max(helical_loop_list), min(helical_loop_list)

    if kinetic_score > max_kinetic_score: return True
    if three_state_indicator > max_three_state_indicator: return True
    #if min(helical_loop_list) < min_helical_loop: return True
    #if max(helical_loop_list) > max_helical_loop: return True

    return False

def RemoveStartCodons(sequence):
    """Removes any start codons from an input sequence."""

    import random
    import re

    regexp_str = "|".join(RBS_Calculator.start_codons)
    find_starts = re.compile(regexp_str)
    matches = find_starts.finditer(sequence.upper())

    new_seq = sequence[:]
    for match in matches:
        start_pos = match.start()

        triplet = []
        triplet.append(random.choice(['A','T','G','C']))
        triplet.append(random.choice(['A','G','C']))
        triplet.append(random.choice(['A','T','C']))

        new_seq = new_seq[0:start_pos] + "".join(triplet) + new_seq[start_pos+3:len(new_seq)+1]

    matches = find_starts.search(new_seq.upper())
    if matches is None:
        return new_seq
    else:
        return RemoveStartCodons(new_seq)

def compnt(nt):
    if (nt.upper() == 'A'): return 'T'
    if (nt.upper() == 'T'): return 'A'
    if (nt.upper() == 'G'): return 'C'
    if (nt.upper() == 'C'): return 'G'

def MCmove_lower_kinetic_score(pre_seq,post_seq,RBS,estimator = None,MaxIters=infinity):
    """Removes long-range base paired nucleotides from an mRNA sequence (pre-seq,CDS,RBS group). Used when generating initial conditions for synthetic RBS sequences."""

    if estimator is None:
        estimator = Run_RBS_Calculator(pre_seq,post_seq,RBS,verbose=False)

    counter=0
    kinetic_score = estimator.kinetic_score_list[0]
    while (counter < MaxIters and kinetic_score > max_kinetic_score):

        structure = estimator.mRNA_structure_list[0]
        mRNA = structure["mRNA"]
        RBS_begin = mRNA.find(RBS)
        RBS_end = RBS_begin + len(RBS)

        #Alter RBS to reduce kinetic score
        #Create a sorted list of bp'd nucleotides with the ones with the highest kinetic score at the top
        ks_list = []
        bp_x = structure["bp_x"]
        bp_y = structure["bp_y"]
        for (nt_x,nt_y) in zip(bp_x,bp_y):
            ks_list.append( (nt_y - nt_x, nt_x, nt_y) )

        dsu_sort(0,ks_list) #Sort max-top according to 1st column: kinetic_score

        #Determine the bp'd nucleotides with the highest kinetic score. Are either located in the RBS?
        #If so, replace them with another random nucleotide
        nucleotides = sets.Set(['A', 'T', 'G', 'C'])

        num_mutations = min(len(ks_list),10)
        for i in range(num_mutations):
            nt_x = ks_list[i][1] - 1 #python index
            nt_y = ks_list[i][2] - 1 #python index

            #nt_x is in the RBS
            if (nt_x >= RBS_begin and nt_x < RBS_end):

                pos = nt_x - RBS_begin
                letter = random.choice(list(nucleotides ^ sets.Set(RBS[pos])))

                #print "Mutating ", RBS[pos], " --> ", letter

                RBS = RBS[0:pos] + letter + RBS[pos+1:len(RBS)+1]


            #nt_y is in the RBS
            elif (nt_y >= RBS_begin and nt_y < RBS_end):
                pos = nt_y - RBS_begin
                letter = random.choice(list(nucleotides ^ sets.Set(RBS[pos])))

                #print "Mutating ", RBS[pos], " --> ", letter

                RBS = RBS[0:pos] + letter + RBS[pos+1:len(RBS)+1]


            elif len(RBS) < RBS_Calculator.cutoff:
                #Insert a nucleotide at the 5' end of the RBS
                letter = random.choice(list(nucleotides))

                #print "Inserting ", letter, " at 5' end"

                RBS = letter + RBS

        RBS = RemoveStartCodons(RBS) #No start codons in RBS!!

        #print "RBS = ", RBS
        estimator = Run_RBS_Calculator(pre_seq,post_seq,RBS,verbose=False)
        kinetic_score = estimator.kinetic_score_list[0]

    return (RBS, estimator)


def MCmove_constrain_helical_loop(pre_seq,post_seq,RBS,estimator):
    """Modifies the mRNA sequence to reduce the size of helical loops."""

    #Alter RBS sequence so that max/min helical loop constraints are valid

    structure = estimator.mRNA_structure_list[0]
    (helical_loop_list,bulge_loop_list,helical_start_ends,bulge_start_ends) = estimator.calc_longest_loop_bulge(structure,True,True,RBS)

    RBS_begin = len(pre_seq)
    RBS_end = RBS_begin + len(RBS)

    #Insert or delete nucleotides to increase/decrease loop length
    for (loop_length,start_end) in zip(helical_loop_list,helical_start_ends):

        if loop_length > max_helical_loop:
            #Choose random nucleotide in loop. Delete it.

            #Identify what part of the loop is in the RBS
            RBS_range = sets.Set(range(RBS_begin+1,RBS_end+1))
            loop_range = sets.Set(range(start_end[0]+1,start_end[1]))
            change_range = list(RBS_range & loop_range) #Intersection

            #print "Loops in RBS: ", change_range

            if len(change_range) > 0:
                pos = random.choice(change_range) - len(pre_seq)
                RBS = RBS[0:pos] + RBS[pos+1:len(RBS)+1] #Delete nucleotide at position pos


        elif loop_length < min_helical_loop:
            #Choose random position in loop and insert a nucleotide before it.
            #Identify what part of the loop is in the RBS
            RBS_range = sets.Set(range(RBS_begin+1,RBS_end+1))
            loop_range = sets.Set(range(start_end[0]+1,start_end[1]))
            change_range = list(RBS_range & loop_range) #Intersection

            #print "Loops in RBS: ", change_range

            if len(change_range) > 0:
                pos = random.choice(change_range) - len(pre_seq)
                letter = random.choice(['A','T','C','G'])
                RBS = RBS[0:pos] + letter + RBS[pos+1:len(RBS)+1]

    #estimator = Run_RBS_Calculator(pre_seq,post_seq,RBS,verbose=False)

    return RBS

def GetInitialRBS(pre_seq,post_seq,dG_target):
    """Generates a random initial condition for designing a synthetic RBS sequence."""

    use_new = True
    Pre_length = 25

    dG_target_nondim = (dG_target - dG_range_high) / (dG_range_low - dG_range_high)
    #0.0: Low expression
    #1.0: High expression

    if     dG_target_nondim <  0.125:
        PchooseSD = 0.50
        Core_length = 4
        max_nonoptimal_spacing = 10
    elif   dG_target_nondim <  0.250:
        PchooseSD = 0.50
        Core_length = 4
        max_nonoptimal_spacing = 10
    elif   dG_target_nondim < 0.5:
        PchooseSD = 0.75
        Core_length = 4
        max_nonoptimal_spacing = 10
    elif   dG_target_nondim < 0.7:
        PchooseSD = 0.75
        Core_length = 4
        max_nonoptimal_spacing = 5
    elif   dG_target_nondim <  0.8:
        PchooseSD = 0.75
        Core_length = 6
        max_nonoptimal_spacing = 5
    elif   dG_target_nondim <  0.9:
        PchooseSD = 0.90
        Core_length = 6
        max_nonoptimal_spacing = 5
    elif   dG_target_nondim <  0.95:
        PchooseSD = 0.90
        Core_length = 8
        max_nonoptimal_spacing = 3
    elif   dG_target_nondim <=  1.00:
        PchooseSD = 1.0
        Core_length = 9
        max_nonoptimal_spacing = 2

    dG_total = dG_range_high + 1
    while dG_total > dG_range_high:
        RBS = Generate_Random_RBS(False, Max_RBS_Length, Pre_length, PchooseSD, Core_length, max_nonoptimal_spacing)
        RBS = RemoveStartCodons(RBS)

        estimator = Run_RBS_Calculator(pre_seq,post_seq,RBS,verbose=False)

        if use_new:
            RBS = MCmove_constrain_helical_loop(pre_seq,post_seq,RBS,estimator)
            (RBS,estimator) = MCmove_lower_kinetic_score(pre_seq,post_seq,RBS,estimator)

        else:
            counter=0
            while dG_total > 0 or calc_constraints(RBS,estimator):
                counter+=1
                RBS = Generate_Random_RBS(False, Max_RBS_Length, Pre_length, PchooseSD, Core_length, max_nonoptimal_spacing)
                RBS = RemoveStartCodons(RBS)

                estimator = Run_RBS_Calculator(pre_seq,post_seq,RBS,verbose=False)
                dG_total = estimator.dG_total_list[0]

        dG_total = estimator.dG_total_list[0]

    return (RBS,estimator)

def Monte_Carlo_Design(pre_seq, post_seq, RBS_init = None, TIR_target = None, dG_target = None, MaxIter = 10000, verbose = False):
    """Master function for designing synthetic RBS sequences without constraints."""

    #Check if dG_total or TIR (translation initiation rate) was specified. If TIR, then convert to dG_total.
    if TIR_target is not None:
        dG_target = RBS_Calculator.RT_eff * (RBS_Calculator.logK - math.log(float(TIR_target)))

    if verbose: print "dG_target = ", dG_target

    #Parameters
    max_init_energy = 10.0 #kcal/mol
    tol = 0.25 #kcal/mol
    annealing_accept_ratios = [0.01, 0.20] #first is min, second is max
    annealing_min_moves = 50
    RT_init = 0.6 #roughly 300K

    weighted_moves = [('insert',0.10),('delete',0.10),('replace',0.80)]

    #Define the energy/cost function based on the dG_target and the other, optional targets
    calc_energy = lambda (dG_total): abs(dG_total - dG_target)

    #If RBS_Init is given, use it. Otherwise, randomly choose one that is a decent starting point.
    if verbose: print "Determining Initial RBS"

    if RBS_init is None:
        (RBS,estimator) = GetInitialRBS(pre_seq,post_seq,dG_target)
    else:
        RBS = RBS_init
        estimator = Run_RBS_Calculator(pre_seq,post_seq,RBS,verbose=False)

    #Initialization
    counter = 0
    accepts = 0
    rejects = 0
    RT = RT_init

    dG_total = estimator.dG_total_list[0]
    energy = calc_energy(dG_total)

    if verbose: print "Initial RBS = ", RBS, " Energy = ", energy
    if verbose: estimator.print_dG(estimator.infinity)

    while energy > tol and counter < MaxIter:

        try:
            counter += 1
            accepted = False

            move = weighted_choice(weighted_moves)

            RBS_new = ''
            if verbose: print "Move #", counter, ": ", move
            if move == 'insert':

                pos = int(random.uniform(0.0,1.0) * len(RBS))
                letter = random.choice(['A', 'T', 'G', 'C'])
                RBS_new = RBS[0:pos] + letter + RBS[pos:len(RBS)]

            if move == 'delete':
                if (len(RBS) > 1):
                    pos = int(random.uniform(0.0,1.0) * len(RBS))
                    RBS_new = RBS[0:pos] + RBS[pos+1:len(RBS)]

            if move == 'replace':
                pos = int(random.uniform(0.0,1.0) * len(RBS))
                letter = random.choice(['A', 'T', 'G', 'C'])
                RBS_new = RBS[0:pos] + letter + RBS[pos+1:len(RBS)]

            RBS_new = RemoveStartCodons(RBS_new)

            if len(RBS_new) > Max_RBS_Length:
                RBS_new = RBS_new[len(RBS_new)-Max_RBS_Length:len(RBS_new)+1]

            estimator = Run_RBS_Calculator(pre_seq,post_seq,RBS_new,verbose=False)

            dG_total = estimator.dG_total_list[0]
            energy_new = calc_energy(dG_total)
            if calc_constraints(RBS_new,estimator):
                energy_new = infinity

            if verbose: print "New energy = ", energy_new

            if energy_new < energy:
                #Accept move immediately
                RBS = RBS_new
                energy = energy_new
                accepted = True
                if verbose: print "Move immediately accepted"
            else:

                ddE = (energy - energy_new)
                Metropolis = math.exp(ddE / RT)
                prob = random.uniform(0.0,1.0)

                if Metropolis > prob:
                    #Accept move based on conditional probability
                    RBS = RBS_new
                    energy = energy_new
                    accepts += 1
                    accepted = True
                    if verbose: print "Move conditionally accepted"
                else:
                    #Reject move
                    rejects += 1
                    if verbose: print "Move rejected"

            if accepted and verbose: estimator.print_dG(estimator.infinity)

            #Simulated annealing control
            if accepts + rejects > annealing_min_moves:

                ratio = float(accepts) / float(accepts + rejects)

                if ratio > annealing_accept_ratios[1]:
                    #Too many accepts, reduce RT
                    RT = RT / 2.0
                    accepts = 0
                    rejects = 0
                    if verbose: print "Accepting too many conditional moves, reducing temperature"

                if ratio < annealing_accept_ratios[0]:
                    #Too many rejects, increase RT
                    RT = RT * 2.0
                    accepts = 0
                    rejects = 0
                    if verbose: print "Rejecting too many conditional moves, increasing temperature"

        except KeyboardInterrupt:
            if verbose: print "Calculating Final State"
            estimator = Run_RBS_Calculator(pre_seq,post_seq,RBS,verbose=False)

            dG_total = estimator.dG_total_list[0]
            return (dG_total, RBS, estimator, counter)

    if verbose: estimator.print_dG(estimator.infinity)

    if verbose: print "Total number of RBS Evaluations: ", num_rbs_calculations

    if TIR_target is not None:

        #Convert back to TIR (Translation Initiation Rate)
        TIR_out = RBS_Calculator.K * math.exp(-dG_total / RBS_Calculator.RT_eff)
        return (TIR_out, RBS, estimator, counter)
    else:
        return (dG_total, RBS, estimator, counter)

def MC_Design_from_file(handle, output, dG_target, verbose = True, **kvars):
    """This function accepts a FASTA formatted file of Pre-sequences and CDSs and generates synthetic RBS sequences with the selected target dG_total. Uses Biopython for reading FASTA files. """

    from Bio import SeqIO

    #Set Defaults
    if not "kinetic_score_max" in kvars.keys(): kvars["kinetic_score_max"] = 0.500

    records = SeqIO.parse(handle,"fasta")

    for record in records:
        record_list.append(record)

    First = True
    counter=0

    assert len(dG_target) == len(record_list), "The length of dG_target should equal the number of designed RBSs in the input file"

    while counter < len(record_list):
        pre_seq = record_list[counter].seq.tostring().upper()
        post_seq = record_list[counter+1].seq.tostring().upper()
        counter += 2

        [dG_total, RBS, estimator] = Monte_Carlo_Design(pre_seq, post_seq, dG_target[counter/2-1], verbose,kvars)

        if verbose: estimator.print_dG(estimator.infinity)
        if verbose: print "Final RBS = ", RBS

        estimator.save_data(output, First)
        if First:
            First = False

        index = estimator.mRNA_rRNA_corrected_structure_list[0]["MinStructureID"]
        estimator.mRNA_rRNA_corrected_structure_list[0].export_PDF(index, "Monte Carlo Result", "MC_" + str(counter/2-1) + "_rRNA" + ".pdf")

        estimator.mRNA_structure_list[0].export_PDF(0, "Monte Carlo Result", "MC_" + str(counter/2-1) + "_mRNA" + ".pdf")

#-------------------------------------------------------------------------------
if __name__ == "__main__":


    pre_RBS = "TTCTAGA"
    post_RBS = "ATGCAGCACGTGTGCAGCACTACAGCGTGTGACGACTACAGCATTCACGACAGTCACATGCAGTTGACAC"
    dG_target = -10.0

    (dG_total, RBS, estimator, iterations) = Monte_Carlo_Design(pre_RBS, post_RBS, RBS_init = None, dG_target = dG_target, MaxIter = 10000, verbose = True)

    print "Finished"
    print "dG_total = ", dG_total
    print "RBS = ", RBS
    print "# iterations = ", iterations




In [ ]:
#@title code of Run_RBS_Design
#!/usr/bin/env python

#This file is part of the Ribosome Binding Site Calculator.

#The Ribosome Binding Site Calculator is free software: you can redistribute it and/or modify
#it under the terms of the GNU General Public License as published by
#the Free Software Foundation, either version 3 of the License, or
#(at your option) any later version.

from RBS_MC_Design import Monte_Carlo_Design
import sys, time

def all(x):
    """The all() function from Python 2.5, for backwards compatibility with previous versions of Python."""

    output = True
    for bool in x:
        if not bool:
            output = False
            break
    return output

def run():

    #Read command line arguments
    input_args = []
    for arg in sys.argv:
        input_args.append(arg)

    if len(input_args) == 4:
        pre_seq = input_args[1]
        post_seq = input_args[2]
        TIR = float(input_args[3])
    else:
        output = "Usage: " + input_args[0] + " [Pre-Sequence] [Post-Sequence] [Translation Initiation Rate]" + "\n"
        return (False,output)

    #Check Sequence and TIR Inputs
    valid_chars = ('A','C','T','G','U','a','c','t','g','u')
    valid_start_codons = ('ATG','AUG','GTG','GUG')

    if TIR <= 0: return (False,"invalid TIR")
    if len(pre_seq) <= 0 or not all([s in valid_chars for s in pre_seq[:]]): return (False,"invalid pre_seq")
    if len(post_seq) <= 0 or not all([s in valid_chars for s in post_seq[:]]) or post_seq[0:3].upper() not in valid_start_codons: return (False,"invalid post_seq")

    total_iterations=0
    MaxIter = 10000
    (TIR_out, RBS_out, estimator, iterations) = Monte_Carlo_Design(pre_seq,post_seq, None, TIR, MaxIter, verbose = False)
    total_iterations+=iterations

    #If RBS is not designed to specs within 10000 iterations, then start from a different initial condition
    while iterations == MaxIter:
        (TIR_out, RBS_out, estimator, iterations) = Monte_Carlo_Design(pre_seq,post_seq, None, TIR, MaxIter, verbose = False)
        total_iterations+=iterations

    output_string = "Program Executed" + "\n" + RBS_out + "\n" + str(TIR_out) + "\n" + str(total_iterations) + "\n"
    return (True,output_string)

def ReadOutput(output):
    import stringIO

    handle = stringIO.stringIO(output)

    for line in handle.readlines():
        if line == "Program Executed":
            RBS_out = handle.readlines()
            TIR_out = float(handle.readlines())
            total_iterations = int(handle.readlines())
            break
        elif line == "Program NOT Executed":
            RBS_out = ""
            TIR_out = -1.0
            total_iterations = 0
            break

    return (RBS_out, TIR_out, total_iterations)

if __name__ == "__main__":

    #Runs RBS Design optimization
    #Prints to standard output -- captured by globusrun interactive and transferred to the submitting computer.
    start = time.clock()

    (success,output) = run()

    end = time.clock()
    time_elapsed = end - start

    if success:
        print output
    else:
        print "Error"
        print output
